In [2]:
# save files
fullproc_r = '/Users/jnaiman/LMM_Figure_Parsing/resources/'

# get title and x/y labels from most frequent words
data_dir = '/Users/jnaiman/LMM_Figure_Parsing/resources/data/'

# save
#fake_figs_dir = '/Users/jnaiman/Downloads/tmp/fake_figs/'
fake_figs_dir = '/Users/jnaiman/Dropbox/JCDL2024/paper_submission/data/'

# only used to re-do inlines
dirs_latex = ['arxiv/9901', 'arxiv_new/2301', 'arxiv_new/2302']
base_dir_arxiv = '/Users/jnaiman/Dropbox/wwt_image_extraction/'

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2 as cv
import pickle
import pandas as pd
#import string
from glob import glob
from mpl_toolkits.axes_grid1 import make_axes_locatable
from PIL import ImageColor
import json

#import spacy

# this is hacktacular from: https://github.com/ReadingTimeMachine/TexSoup
# and needs to be updated
# tex_lib = '/Users/jnaiman/TexSoup/'
# from sys import path
# path.append(tex_lib)
# import TexSoup
# from TexSoup import preprocessing, postprocess

#from nltk.corpus import stopwords
import time

# from matplotlib import font_manager
# plt.rcParams['text.usetex'] = True
import matplotlib as mpl
mpl.rcParams['text.usetex'] = True
mpl.rcParams['text.latex.preamble'] = r'\usepackage{amsmath} \usepackage{amssymb}' #for \text command

#nlp = spacy.load("en_core_web_sm")

# written libs
from importlib import reload # debug
import synthetic_fig_utils # debug
reload(synthetic_fig_utils) # debug
from synthetic_fig_utils import subset_by_percent, \
 get_nrows_and_ncols, normalize_params_prob, get_ticks, get_titles_or_labels, \
 get_font_info
 #get_data, make_plot
#get_line_styles, \

import plot_parameters


# import distribution_utils
# reload(distribution_utils)
# from distribution_utils import get_random_data, get_random, \
#    get_linear, get_linear_data, get_data

import plot_utils
reload(plot_utils)
from plot_utils import get_contour_plot, get_histogram_plot, \
   get_line_plot, get_scatter_plot, make_plot#, get_line_plot

import pixel_location_utils
reload(pixel_location_utils)
from pixel_location_utils import get_errorbar_pixels, get_data_pixel_locations

import text_utils
reload(text_utils)
from text_utils import get_popular_nouns, get_inline_math

# create a bunch of fake figures
reload(synthetic_fig_utils)
from synthetic_fig_utils import normalize_params_prob
reload(plot_parameters)
from plot_parameters import plot_types_params, panel_params, \
  title_params, xlabel_params, \
  ylabel_params, aspect_fig_params, dpi_params, tight_layout_params, \
  fontsizes, base

import data_utils
reload(data_utils)
from data_utils import get_data, NumpyEncoder

use_uniques = True # use unique inlines
verbose = True

In [4]:
#print('... for the plot in the', plot_index_to_words([0,1]), 'panel ...')

In [5]:
# get fonts -- see "cnn_create_synthetic_ticks" in FullProcess
dfont = pd.read_csv(fullproc_r + 'fonts.csv')
font_names = dfont['font name'].values

# for plot styles
plot_styles = plt.style.available

In [6]:
# get words
popular_nouns = get_popular_nouns(data_dir)

16253 343492
shape of output= (5097, 5)
shape of output= (36332, 5)


Get inline math:

In [7]:
inlines = get_inline_math(base_dir_arxiv, dirs_latex, fullproc_r,
                          recreate_inlines=False,
                         use_uniques=use_uniques)

number of inlines =  33201


## Update plotting params

In [8]:
plot_params = plot_types_params.copy()

In [9]:
#plot_params['histogram']

### Line plots, Linear distributions

In [10]:
linestyles = ['-', '--', ':'] # only use a subset of the linestyles

plot_params_line = {'line':plot_params['line'].copy()}

plot_params_line['line']['npoints'] = {'min':10, 'max':100}
plot_params_line['line']['nlines'] = {'min':1, 'max':10}

# just linear, random
plot_params_line['line']['distribution']['random']['prob'] = 1

# gaussian mixture model
plot_params_line['line']['distribution']['gmm']['prob'] = 1
plot_params_line['line']['distribution']['gmm']['histogram as line']['prob'] = 1
plot_params_line['line']['distribution']['gmm']['nclusters'] = {'min': 1, 'max': 5}
plot_params_line['line']['distribution']['gmm']['nsamples'] = {'min': 10, 'max': 50}

# linear plots
plot_params_line['line']['distribution']['linear']['prob'] = 1
# noise
#plot_params_line['line']['distribution']['linear']['noise'] = (0.25, 0.25)


plot_params_line, panel_params, \
  title_params, xlabel_params, \
  ylabel_params = normalize_params_prob(plot_params_line.copy(), panel_params, 
                                        title_params, xlabel_params, 
                                        ylabel_params)

plot_params_line

title_params capatilize did not add to 1! total = 1.2999999999999998
renormalizing...
now:  {'none': 0.07692307692307694, 'first': 0.3846153846153847, 'all': 0.5384615384615385}
xlabel_params capatilize did not add to 1! total = 1.2999999999999998
renormalizing...
now:  {'none': 0.07692307692307694, 'first': 0.3846153846153847, 'all': 0.5384615384615385}
ylabel_params capatilize did not add to 1! total = 1.2999999999999998
renormalizing...
now:  {'none': 0.07692307692307694, 'first': 0.3846153846153847, 'all': 0.5384615384615385}
plot_types_params['line']['distribution'] probabilities did not add to 1! total = 3
renormalizing...
now:  [0.3333333333333333, 0.3333333333333333, 0.3333333333333333]


{'line': {'prob': 1,
  'npoints': {'min': 10, 'max': 100},
  'line thick': {'min': 1, 'max': 5},
  'nlines': {'min': 1, 'max': 10},
  'xmin': -10000,
  'xmax': 10000,
  'ymin': -10000,
  'ymax': 10000,
  'prob same x': 0.1,
  'markers': {'prob': 0.5, 'size': {'min': 1, 'max': 20}},
  'error bars': {'elinewidth': {'min': 1, 'max': 5},
   'x': {'prob': 0.25, 'size': {'min': 0.01, 'max': 0.1}},
   'y': {'prob': 0.25, 'size': {'min': 0.01, 'max': 0.1}}},
  'distribution': {'random': {'prob': 0.3333333333333333},
   'linear': {'prob': 0.3333333333333333,
    'intersect': (-100, 100),
    'slope': (-5, 5),
    'noise': (0, 0.25)},
   'gmm': {'prob': 0.3333333333333333,
    'histogram as line': {'prob': 1, 'factor': 1000},
    'nclusters': {'min': 1, 'max': 5},
    'nsamples': {'min': 10, 'max': 50},
    'cluster std': {'min': -2, 'max': 0.5},
    'noise': {'min': 0.05, 'max': 0.25}}}}}

In [11]:
def log_scale_ax(scale_exp = {'min':0.01, 'max':5.0}, 
                 scale_factor = {'min':0.1, 'max':10}, 
                 npoints=1,
                 verbose=False):
    """
    npoints : how many objects do we want?  only 1 is implemented right now
    """
    if npoints != 1:
        print('only npoints=1 is implemented!')
        import sys; sys.exit()
    scale_input = np.random.uniform(low=scale_exp['min'], high=scale_exp['max'])
    n = np.random.exponential(scale_input)
    scale = 10**n
    scale_factor_low = np.random.uniform(low=scale_factor['min'], high=scale_factor['max'])
    xmin = scale - scale_factor_low*scale
    scale_factor_high = np.random.uniform(low=scale_factor['min'], high=scale_factor['max'])
    xmax = scale + scale_factor_high*scale
    if verbose: print(scale_input, n, scale, xmin,xmax)
    return xmin,xmax

In [12]:
#plot_types_params
plot_types = []
for p,v in plot_types_params.items():
    plot_types.append(p)
plot_types

['line', 'histogram', 'scatter', 'contour']

In [13]:
plot_types_qa = ['line', 'scatter', 'histogram']

## Generate "toy" data to test QA pairs


In [ ]:
plot_params_here = plot_params_line.copy()
ifigure = 0

datas_all = {} # for multiples
plt.close('all')

if True:

    ######### pick things ########
    # figure
    color_map = 'gray' # hack#np.random.choice(plt.colormaps())
    #plt.set_cmap(color_map)
    
    npanels, panel_style, nrows, ncols = get_nrows_and_ncols(panel_params)
    ####npanels, panel_style,nrows,ncols = 1, 'horizontal', 1,1 # HACK
    plot_style = np.random.choice(plot_styles)
    
    aspect_fig = np.random.uniform(low=aspect_fig_params['min'], 
                                   high=aspect_fig_params['max'])
    # rows and cols 
    aspect_fig *= nrows/ncols
    dpi = int(np.random.uniform(low=dpi_params['min'], high=dpi_params['max']))
    # tight layout or not
    print('panel style:', panel_style, 'nrows,ncols=', (nrows,ncols), 
          'aspect=', aspect_fig, 'dpi=', dpi)
    
    tight_layout = True


    # get all font stuffs
    title_fontsize, xlabel_fontsize, ylabel_fontsize, \
       xlabel_ticks_fontsize, ylabel_ticks_fontsize, \
                           csfont = get_font_info(fontsizes, font_names)
    #print("ALL IS WELL HERE 1")

    # plot types?
    success = False
    start_time = time.time()
    while not success:
        #with plt.style.context(plot_style):
        if True:
            plt.style.use(plot_style)
            plt.set_cmap(color_map) 
            #plt.rcParams['image.cmap'] = 'gray' #color_map
            if tight_layout:
                fig,axes = plt.subplots(nrows,ncols,figsize=(base*nrows, base*aspect_fig*ncols), 
                                        dpi=dpi,layout='tight')
            else:
                fig,axes = plt.subplots(nrows,ncols,figsize=(base*nrows, base*aspect_fig*ncols), 
                                        dpi=dpi)
                
            if npanels == 1:
                axes = [axes]
                plot_inds = [(0,0)] # i,j
            else: # flatten, for now
                # create inds
                if len(axes.shape) > 1: # 2d
                    ashape = np.array(axes.shape).copy()
                else:
                    ashape = [nrows, ncols]

                plot_inds = np.empty(shape=(ashape[0], ashape[1],2), dtype=int)
                for i in range(ashape[0]):
                    for j in range(ashape[1]):
                        plot_inds[i,j][0] = i
                        plot_inds[i,j][1] = j
                plot_inds = plot_inds.reshape((ashape[0]*ashape[1],-1))
                axes = axes.flatten()
            

            #**HERE** have to save index
                    
            ######### Generate plot data #########
            #print('here 1')
            
            choices = []; probs = []
            for k,v in plot_params_here.items():
                choices.append(k)
                probs.append(v['prob'])
        
            data_for_plots = []
            plot_types = []
            data_from_plots = []
            titles = []; xlabels = []; ylabels = []; cbars = []
            end_time = time.time()
            #try:
            if True:
                for iplot, ax in enumerate(axes): # pick seperate scales for all
                    plot_params_here_ax = plot_params_here.copy()
                    start_time = time.time()
                    plot_type = np.random.choice(choices, p=probs)
                    print('PLOT TYPE:', plot_type)
                    # get distribution type
                    dist_params = plot_params_here_ax[plot_type]['distribution'] 
                    choices_d = []; probs_d = []
                    for k,v in dist_params.items():
                        choices_d.append(k)
                        probs_d.append(v['prob'])

                    distribution_type = np.random.choice(choices_d, p=probs_d)
                    print('  Distribution Type:', distribution_type)
                    
                    xmin,xmax = log_scale_ax()
                    plot_params_here_ax[plot_type]['xmin']=xmin
                    plot_params_here_ax[plot_type]['xmax']=xmax
                    if plot_type == 'line' or plot_type == 'scatter' or plot_type == 'contour':
                        ymin,ymax = log_scale_ax()
                        plot_params_here_ax[plot_type]['ymin']=ymin
                        plot_params_here_ax[plot_type]['ymax']=ymax
                    if plot_type == 'scatter' or plot_type == 'contour': 
                        cmin,cmax = log_scale_ax()
                        plot_params_here_ax[plot_type]['color']['min']=cmin
                        plot_params_here_ax[plot_type]['color']['max']=cmax
                    
                    #print('here 2')
                    data_for_plot = get_data(plot_params_here_ax[plot_type],
                                    plot_type=plot_type,
                                            distribution=distribution_type)
                    end_time = time.time()
                
                    ######### PLOT ############
                    start_time = time.time()
                    #print('here 3')
                    data_from_plot, ax = make_plot(plot_params_here_ax[plot_type], data_for_plot, 
                                         ax, plot_type=plot_type, linestyles=linestyles)#, plot_style=plot_style)
                    #import sys; sys.exit()
                    end_time = time.time()
                    #print('here 4')
    
                    # set ticksizes
                    ax.tick_params(axis='x', which='major', labelsize=xlabel_ticks_fontsize, labelfontfamily=csfont['fontname'])
                    ax.tick_params(axis='y', which='major', labelsize=ylabel_ticks_fontsize, labelfontfamily=csfont['fontname'])
    
                    start_time = time.time()
                    p = np.random.uniform(0,1)
                    if p < title_params['prob']:
                        title_words = get_titles_or_labels(popular_nouns, title_params['capitalize'],
                                                 title_params['equation'], inlines,
                                                 nwords=np.random.randint(low=title_params['n words']['min'],
                                                                          high=title_params['n words']['max']+1))
                        title = ax.set_title(title_words, fontsize = title_fontsize, **csfont)
                    else:
                        title = ''
                        
                    xlabel_words = get_titles_or_labels(popular_nouns, xlabel_params['capitalize'],
                                                 xlabel_params['equation'], inlines,
                                                 nwords=np.random.randint(low=xlabel_params['n words']['min'],
                                                                          high=xlabel_params['n words']['max']+1))
                    xlabel = ax.set_xlabel(xlabel_words, fontsize=xlabel_fontsize, **csfont)
                    ylabel_words = get_titles_or_labels(popular_nouns, ylabel_params['capitalize'],
                                                 ylabel_params['equation'], inlines,
                                                 nwords=np.random.randint(low=ylabel_params['n words']['min'],
                                                                          high=ylabel_params['n words']['max']+1))
                    ylabel = ax.set_ylabel(ylabel_words, fontsize=ylabel_fontsize, **csfont)
                    end_time = time.time()
            
                    # save
                    data_for_plots.append(data_for_plot)
                    plot_types.append(plot_type)
                    data_from_plots.append(data_from_plot)
                    titles.append(title)
                    xlabels.append(xlabel)
                    ylabels.append(ylabel)
    
                    if plot_type == 'scatter': # or plot_type == 'contour':
                        if 'color bar' in data_from_plot:
                            side = data_from_plot['color bar params']['side']
                            if side == 'top' or side == 'bottom':
                                orientation = 'horizontal'
                            else:
                                orientation = 'vertical'
    
                            cbar = fig.colorbar(data_from_plot['data'], 
                                         cax=data_from_plot['color bar'], 
                                         orientation=orientation)
                            cbars.append(cbar)

                    if plot_type == 'contour':
                        if 'color bar' in data_from_plot:
                            side = data_from_plot['color bar params']['side']
                            if side == 'top' or side == 'bottom':
                                orientation = 'horizontal'
                            else:
                                orientation = 'vertical'
                    
                            if 'image' in data_from_plot['data']: # select correct colorbar to use
                                datac = data_from_plot['data']['image']
                            else:
                                datac = data_from_plot['data']['contour']
                                            
                            cbar = fig.colorbar(datac, 
                                         cax=data_from_plot['color bar'], 
                                         orientation=orientation)
                            cbars.append(cbar)

    
                #if not save_smalls:
                plt.set_cmap(color_map) # do again
                fig.savefig(fake_figs_dir + 'Picture' + str(ifigure+1) + '.png', dpi=dpi, bbox_inches='tight')
                print('saved:', fake_figs_dir + 'Picture' + str(ifigure+1) + '.png')
                # else:
                #     fig.savefig(save_small_dir + 'Picture' + str(ifigure+1) + '.png', dpi=dpi)
                #     print('saved:', save_small_dir + 'Picture' + str(ifigure+1) + '.png')
                    
                success = True
            #except Exception as e:
            else:
                print('[ERROR]:')
                plt.close(fig)
                print('issue with plotting, trying again')
                print(e)
                if 'unknown color specifier' in str(e):
                    print(e)
                    import sys; sys.exit()
                if 'At least one value in the dash list must be positive' in str(e):
                    print(e)
                    print(data_from_plots)
    
    
    ####### end of plotting ############
    # try the whole thing again
    width, height = fig.canvas.get_width_height()
    # save data
    datas = {}
    # figure datas
    datas['figure'] = {'dpi':dpi, 'base':base, 'aspect ratio': aspect_fig, 
                          'nrows':nrows, 'ncols':ncols, 
                         'plot style':plot_style, 
                         'color map':color_map,
                         'title fontsize':title_fontsize, 
                         'xlabel fontsize':xlabel_fontsize,
                         'ylabel fontsize':ylabel_fontsize, 
                      'plot indexes':plot_inds}
    
    # now, get data things
    for iplot, ax in enumerate(axes):
        ###### get data from plot ######
        data_from_plot = data_from_plots[iplot]
        data_for_plot = data_for_plots[iplot]
        plot_type = plot_types[iplot]
        title = titles[iplot]
        xlabel = xlabels[iplot]
        ylabel = ylabels[iplot]
    
        data_pixels = get_data_pixel_locations(data_from_plot, plot_type, ax, width, height)
    
        # bounding box of square
        bbox = ax.get_position() # Bbox(x0, y0, x1, y1)
        xpix1 = np.array([bbox.x0,bbox.x1])
        ypix1 = np.array([bbox.y0,bbox.y1])
        xpix1 *= width
        ypix1 *= height
        
        # x-tick locations
        xticks = get_ticks(ax.get_xticklabels(), ax.get_xticklines())
    
        # y-tick locations
        yticks = get_ticks(ax.get_yticklabels(), ax.get_yticklines())
        
        # for colorbars
        colorbar_ticks = []
        if 'color bar' in data_from_plots[iplot]:
            colorbar = data_from_plots[iplot]['color bar']
            if data_from_plots[iplot]['color bar params']['side'] == 'left' \
               or data_from_plots[iplot]['color bar params']['side'] == 'right':
                ticks = colorbar.get_yticklabels()
                tick_locs = colorbar.get_yticklines(minor=False)
            else:
                ticks = colorbar.get_xticklabels()
                tick_locs = colorbar.get_xticklines(minor=False)
            colorbar_ticks = get_ticks(ticks, tick_locs)

        # title
        # Get the bounding box of the title in display space
        if title != '':
            title_bbox = title.get_window_extent()
            title_words = title.get_text()
        else:
            title_bbox = -1
            title_words = ''
    
        # xlabel
        xlabel_bbox = xlabel.get_window_extent()
        xlabel_words = xlabel.get_text()
        # ylabel
        ylabel_bbox = ylabel.get_window_extent()
        ylabel_words = ylabel.get_text()
    
    
        ########## save data ##############

        # line plot 
        plot_name = 'plot' + str(iplot) 
        datas[plot_name] = {}
        # line plot type
        datas[plot_name]['type'] = plot_type # tag for kind of plot
        datas[plot_name]['distribution'] = distribution_type
        datas[plot_name]['data'] = data_for_plot
        datas[plot_name]['data pixels'] = data_pixels
        datas[plot_name]['data from plot'] = json.loads(json.dumps(data_from_plot, cls=NumpyEncoder))
        if (plot_type == 'scatter' or plot_type == 'contour') and 'color bar' in data_from_plots[iplot]:
            #print('yes indeed')
            w = data_from_plots[iplot]['color bar'].get_window_extent()
            datas[plot_name]['color bar'] = {'xmin':w.x0,'ymin':w.y0,
                                             'xmax':w.x1,'ymax':w.y1, 
                                             'params':data_from_plot['color bar params']}
        xtmp = []
        for xt in xticks:
            l = {'data':xt[0], 'xmin': xt[1], 
                 'ymin': xt[2], 
                 'xmax':xt[3], 'ymax':xt[4],
                 'tx':xt[5], 'ty':xt[6]}
            xtmp.append(l)
        datas[plot_name]['xticks'] = xtmp.copy()
        # 
        xtmp = []
        for xt in yticks:
            l = {'data':xt[0], 'xmin': xt[1], 
                 'ymin': xt[2], 
                 'xmax':xt[3], 'ymax':xt[4], 
                'tx':xt[5], 'ty':xt[6]}
            xtmp.append(l)
        datas[plot_name]['yticks'] = xtmp.copy()
        if len(colorbar_ticks) > 0:
            xtmp = []
            for xt in colorbar_ticks:
                l = {'data':xt[0], 'xmin': xt[1], 
                     'ymin': xt[2], 
                     'xmax':xt[3], 'ymax':xt[4], 
                    'tx':xt[5], 'ty':xt[6]}
                xtmp.append(l)
            datas[plot_name]['color bar ticks'] = xtmp.copy()
            
        # axis box
        datas[plot_name]['square'] = {'xmin':xpix1[0], 'ymin':ypix1[0], 
                                         'xmax':xpix1[1], 'ymax':ypix1[1]}
        # title
        if title_bbox != -1:
            datas[plot_name]['title'] = {'xmin':title_bbox.x0, 'ymin':title_bbox.y0, 
                                            'xmax':title_bbox.x1, 'ymax':title_bbox.y1,
                                           'words':title_words}
        else:
            pass
        datas[plot_name]['xlabel'] = {'xmin':xlabel_bbox.x0, 'ymin':xlabel_bbox.y0, 
                                        'xmax':xlabel_bbox.x1, 'ymax':xlabel_bbox.y1,
                                       'words':xlabel_words}
        datas[plot_name]['ylabel'] = {'xmin':ylabel_bbox.x0, 'ymin':ylabel_bbox.y0, 
                                        'xmax':ylabel_bbox.x1, 'ymax':ylabel_bbox.y1,
                                       'words':ylabel_words}

    # # dump full data
    # dumped = json.dumps(datas, cls=NumpyEncoder)
    # with open(fake_figs_dir + 'Picture' + str(ifigure+1) + '_fullData.json', 'w') as f:
    #     json.dump(dumped, f)
            
    # for full thing
    #datas_all[ifigure] = datas.copy()
    plt.close(fig)
print('DONE!')
# back to basics
plt.style.use('default')

selected npanels: 14
panel style: squarish nrows,ncols= (5, 3) aspect= 2.3092128250060755 dpi= 90
PLOT TYPE: line
  Distribution Type: linear
PLOT TYPE: line
  Distribution Type: random
PLOT TYPE: line
  Distribution Type: linear
PLOT TYPE: line
  Distribution Type: random
PLOT TYPE: line
  Distribution Type: random
PLOT TYPE: line
  Distribution Type: gmm
PLOT TYPE: line
  Distribution Type: gmm
PLOT TYPE: line
  Distribution Type: linear
PLOT TYPE: line
  Distribution Type: gmm
PLOT TYPE: line
  Distribution Type: linear
PLOT TYPE: line
  Distribution Type: random
PLOT TYPE: line
  Distribution Type: gmm
PLOT TYPE: line
  Distribution Type: random
PLOT TYPE: line
  Distribution Type: linear


In [ ]:
fig

## Generate QA Pairs from saved data

Inspiration comes from:
* cPAPERS: http://arxiv.org/abs/2406.08398

### Basics: L1

#### Figure-level questions
1. How many plots

In [16]:
# create qa pairs
qa_pairs = {}
# question levels    
qa_pairs['Level 1'] = {}
qa_pairs['Level 2'] = {}
qa_pairs['Level 3'] = {}
qa_pairs['Level 1']['Figure-level questions'] = {} # Figure-level questions
qa_pairs['Level 1']['Plot-level questions'] = {}
qa_pairs['Level 2']['Plot-level questions'] = {}
qa_pairs['Level 3']['Plot-level questions'] = {}

In [17]:
#for k,data in datas_all.items():
def q1(data, qa_pairs, return_qa=True, verbose=True):
    q1 = 'How many panels are in this figure?'
    q1 += ' You are a helpful assistant, please format the output as a json as {"nrows":"", "ncols":""} to store the number of rows and columns.'
    a1 = {"nrows":data['figure']['nrows'], "ncols":data['figure']['ncols']}
    if verbose:
        print('QUESTION:', q1)
        print('ANSWER:', a1)
        print('')
    # add to pairs
    if return_qa: 
        qa_pairs['Level 1']['Figure-level questions']['rows/columns'] = {'Q':q1, 'A':a1}
    return qa_pairs

2. What is the plot style?

In [18]:
#for k,data in datas_all.items():
def q2(data, qa_pairs, return_qa=True, verbose=True):
    q2 = 'Assuming this is a figure made with matplotlib in Python, what is the plot style used?  Examples of plotting styles are "classic" or "ggplot".'
    q2 += ' You are a helpful assistant, please format the output as a json as {"plot style":""} to store the matplotlib plotting style used in the figure.'
    a2 = {"plot style":data['figure']['plot style']}
    if verbose:
        print('QUESTION:', q2)
        print('ANSWER:', a2)
        print('')
    # add to pairs
    if return_qa: qa_pairs['Level 1']['Figure-level questions']['plot style'] = {'Q':q2, 'A':a2}
    return qa_pairs

3. Color map

In [19]:
#for k,data in datas_all.items():
def q3(data, qa_pairs, return_qa=True, verbose=True):
    q2 = 'Assuming this is a figure made with matplotlib in Python, what is the colormap that was used?  Examples of matplotlib colormaps are "rainbow" or "Reds".'
    q2 += ' You are a helpful assistant, please format the output as a json as {"colormap":""} to store the matplotlib colormap used in the figure.'
    a2 = {"colormap":data['figure']['color map']}
    if verbose:
        print('QUESTION:', q2)
        print('ANSWER:', a2)
        print('')
    # add to pairs
    if return_qa: qa_pairs['Level 1']['Figure-level questions']['colormap'] = {'Q':q2, 'A':a2, 
                                                                 'notes':"Some of the plot styles don't allow for updates to the colormap for REASONS, so just keep that in mind."}
    return qa_pairs

4. Aspect ratio

In [20]:
#for k,data in datas_all.items():
def q4(data, qa_pairs, return_qa=True, verbose=True):
    q1 = 'What is the aspect ratio of this figure?'
    q1 += ' You are a helpful assistant, please format the output as a json as {"aspect ratio":""} to store the aspect ratio of the plot.'
    a1 = {"aspect ratio":data['figure']['aspect ratio']}
    if verbose:
        print('QUESTION:', q1)
        print('ANSWER:', a1)
        print('')
    # add to pairs
    if return_qa: qa_pairs['Level 1']['Figure-level questions']['aspect ratio'] = {'Q':q1, 'A':a1}
    return qa_pairs

5. DPI

In [21]:
def dpi(data, qa_pairs, return_qa=True, verbose=True):
    q1 = 'What is the DPI (dots per square inch) of this figure?'
    q1 += ' You are a helpful assistant, please format the output as a json as {"dpi":""} to store the DPI of the plot.'
    a1 = {"dpi":data['figure']['dpi']}
    if verbose:
        print('QUESTION:', q1)
        print('ANSWER:', a1)
        print('')
    # add to pairs
    if return_qa: 
        qa_pairs['Level 1']['Figure-level questions']['dpi'] = {'Q':q1, 'A':a1}
        return qa_pairs

In [22]:
dpi(datas, qa_pairs, return_qa=False)

QUESTION: What is the DPI (dots per square inch) of this figure? You are a helpful assistant, please format the output as a json as {"dpi":""} to store the DPI of the plot.
ANSWER: {'dpi': 402}



#### Specific to individual plots

Title/axis labels:

In [23]:
def q_plot_titles(data, qa_pairs, return_qa=True, verbose=True):
    q = 'What are the titles for each figure panel?'
    q += ' You are a helpful assistant, please format the output as a json as {"titles":[]}, where the list is a list of strings of all of the titles.  '
    q += 'If there is a single plot, this should be one element in this list, and if there are multiple plots the list should be in row-major (C-style) order.'
    q += " If a plot does not have a title, then denote this by an empty string in the list.  Please format any formulas in the title in a Python LaTeX string (for example 'Light $\\\\alpha$'). "
    la = []
    for k,v in data.items():
        if 'plot' in k: # is a plot
            if 'title' not in v:
                la.append("")
            else:
                la.append(v['title']['words'])
    a = {"title":la}
    if verbose:
        print('QUESTION:', q)
        print('ANSWER:', a)
    if return_qa: 
        qa_pairs['Level 1']['Plot-level questions']['titles'] = {'Q':q, 'A':a}
        return qa_pairs

In [24]:
q_plot_titles(datas, qa_pairs, return_qa=False)

QUESTION: What are the titles for each figure panel? You are a helpful assistant, please format the output as a json as {"titles":[]}, where the list is a list of strings of all of the titles.  If there is a single plot, this should be one element in this list, and if there are multiple plots the list should be in row-major (C-style) order. If a plot does not have a title, then denote this by an empty string in the list.  Please format any formulas in the title in a Python LaTeX string (for example 'Light $\\alpha$'). 
ANSWER: {'title': ['COLOR ZONES', 'QUASARS CALIBRATOR VELOCITY BIAS SOURCES']}


In [25]:
def q_plot_xlabels(data, qa_pairs, return_qa=True, verbose=True):
    q = 'What are the x-axis titles for each figure panel?'
    q += ' You are a helpful assistant, please format the output as a json as {"xlabels":[]}, where the list is a list of strings of all of the x-axis titles.  '
    q += 'If there is a single plot, this should be one element in this list, and if there are multiple plots the list should be in row-major (C-style) order.'
    q += " If a plot does not have an x-axis title, then denote this by an empty string in the list.  "
    q += "Please format any formulas in the title in a Python LaTeX string (for example 'Light $\\\\alpha$'). "
    la = []
    for k,v in data.items():
        if 'plot' in k: # is a plot
            if 'xlabel' not in v:
                la.append("")
            else:
                la.append(v['xlabel']['words'])
    a = {"xlabels":la}
    if verbose:
        print('QUESTION:', q)
        print('ANSWER:', a)
    if return_qa: 
        qa_pairs['Level 1']['Plot-level questions']['xlabels'] = {'Q':q, 'A':a}
        return qa_pairs

In [26]:
q_plot_xlabels(datas, qa_pairs, return_qa=False)

QUESTION: What are the x-axis titles for each figure panel? You are a helpful assistant, please format the output as a json as {"xlabels":[]}, where the list is a list of strings of all of the x-axis titles.  If there is a single plot, this should be one element in this list, and if there are multiple plots the list should be in row-major (C-style) order. If a plot does not have an x-axis title, then denote this by an empty string in the list.  Please format any formulas in the title in a Python LaTeX string (for example 'Light $\\alpha$'). 
ANSWER: {'xlabels': ['CONCLUSIONS $F_+^2 + F_\\times^2=1$  TRAJECTORY', 'sne']}


In [27]:
def q_plot_ylabels(data, qa_pairs, return_qa=True, verbose=True):
    q = 'What are the y-axis titles for each figure panel?'
    q += ' You are a helpful assistant, please format the output as a json as {"ylabels":[]}, where the list is a list of strings of all of the y-axis titles.  '
    q += 'If there is a single plot, this should be one element in this list, and if there are multiple plots the list should be in row-major (C-style) order.'
    q += " If a plot does not have an y-axis title, then denote this by an empty string in the list.  "
    q += "Please format any formulas in the title in a Python LaTeX string (for example 'Light $\\\\alpha$'). "
    la = []
    for k,v in data.items():
        if 'plot' in k: # is a plot
            if 'ylabel' not in v:
                la.append("")
            else:
                la.append(v['ylabel']['words'])
    a = {"ylabels":la}
    if verbose:
        print('QUESTION:', q)
        print('ANSWER:', a)
    if return_qa: 
        qa_pairs['Level 1']['Plot-level questions']['ylabels'] = {'Q':q, 'A':a}
        return qa_pairs

In [28]:
q_plot_ylabels(datas, qa_pairs, return_qa=False)

QUESTION: What are the y-axis titles for each figure panel? You are a helpful assistant, please format the output as a json as {"ylabels":[]}, where the list is a list of strings of all of the y-axis titles.  If there is a single plot, this should be one element in this list, and if there are multiple plots the list should be in row-major (C-style) order. If a plot does not have an y-axis title, then denote this by an empty string in the list.  Please format any formulas in the title in a Python LaTeX string (for example 'Light $\\alpha$'). 
ANSWER: {'ylabels': ['PRIORS', 'COLLECTION']}


x/y tick label values

In [29]:
def ticklabels(data, qa_pairs, tick_type='x', return_qa=True, verbose=True):
    q = 'What are the values for each of the tick marks on the '+tick_type+'-axis?'
    q += ' You are a helpful assistant, please format the output as a json as {"'+tick_type+'tick values":[[]]}, where each element of the outer list refers to a single panel, '
    q += 'and each inner list is a list of the '+tick_type+'-axis tick mark values. '
    q += 'If there is a single plot, this should be one element in the outer list, and if there are multiple plots the outer list should be in row-major (C-style) order.'
    q += " If a plot does not have any "+tick_type+"-axis tick values, then denote this by an empty string in the inner list.  "
    q += "Please format any formulas in the "+tick_type+"-axis tick values in a Python LaTeX string (for example 'Light $\\\\alpha$'). "
    la = []
    for k,v in data.items():
        if 'plot' in k: # is a plot
            if tick_type+'ticks' not in v:
                la.append("")
            else:
                t = []
                for l in v[tick_type+'ticks']:
                    t.append(l['data'])
                la.append(t)
    a = {tick_type+"tick values":la}
    if verbose:
        print('QUESTION:', q)
        print('ANSWER:', a)
    if return_qa: 
        qa_pairs['Level 1']['Plot-level questions'][tick_type+'tick values'] = {'Q':q, 'A':a}
        return qa_pairs

In [30]:
ticklabels(datas, qa_pairs, tick_type='x', return_qa=False)

QUESTION: What are the values for each of the tick marks on the x-axis? You are a helpful assistant, please format the output as a json as {"xtick values":[[]]}, where each element of the outer list refers to a single panel, and each inner list is a list of the x-axis tick mark values. If there is a single plot, this should be one element in the outer list, and if there are multiple plots the outer list should be in row-major (C-style) order. If a plot does not have any x-axis tick values, then denote this by an empty string in the inner list.  Please format any formulas in the x-axis tick values in a Python LaTeX string (for example 'Light $\\alpha$'). 
ANSWER: {'xtick values': [['$\\mathdefault{−4000}$', '$\\mathdefault{−2000}$', '$\\mathdefault{0}$', '$\\mathdefault{2000}$'], ['$\\mathdefault{−2}$', '$\\mathdefault{0}$', '$\\mathdefault{2}$', '$\\mathdefault{4}$']]}


In [31]:
ticklabels(datas, qa_pairs, tick_type='y', return_qa=False)

QUESTION: What are the values for each of the tick marks on the y-axis? You are a helpful assistant, please format the output as a json as {"ytick values":[[]]}, where each element of the outer list refers to a single panel, and each inner list is a list of the y-axis tick mark values. If there is a single plot, this should be one element in the outer list, and if there are multiple plots the outer list should be in row-major (C-style) order. If a plot does not have any y-axis tick values, then denote this by an empty string in the inner list.  Please format any formulas in the y-axis tick values in a Python LaTeX string (for example 'Light $\\alpha$'). 
ANSWER: {'ytick values': [['$\\mathdefault{−3.8}$', '$\\mathdefault{−3.6}$', '$\\mathdefault{−3.4}$', '$\\mathdefault{−3.2}$', '$\\mathdefault{−3.0}$', '$\\mathdefault{−2.8}$', '$\\mathdefault{−2.6}$', '$\\mathdefault{−2.4}$'], ['$\\mathdefault{−15.0}$', '$\\mathdefault{−12.5}$', '$\\mathdefault{−10.0}$', '$\\mathdefault{−7.5}$', '$\\m

Plot types:

In [32]:
def q_plot_types(data, qa_pairs, plot_types, return_qa=True, use_list=True, verbose=True):
    q = 'What are the plot types for each panel in the figure? '
    q += 'You are a helpful assistant, please format the output as a json as {"plot types":[]}, where each element of the list refers to the plot type of a single panel. '
    q += 'If there is a single plot, this should be one element in this list, and if there are multiple plots the list should be in row-major (C-style) order. '
    if use_list:
        q += 'Please choose each plot type from the following list: ['
        for pt in plot_types:
            q += pt + ', '
        q = q[:-2] # take off the last bit
        q += '].'
    la = []
    for k,v in data.items():
        if 'plot' in k: # is a plot
            if 'type' not in v:
                la.append("")
            else:
                la.append(v['type'])
    a = {"plot types":la}
    if verbose:
        print('QUESTION:', q)
        print('ANSWER:', a)
    if return_qa: 
        qa_pairs['Level 1']['Plot-level questions']['plot types'] = {'Q':q, 'A':a}
        return qa_pairs

In [33]:
q_plot_types(datas, qa_pairs, plot_types_qa, return_qa=False, use_list=True)

QUESTION: What are the plot types for each panel in the figure? You are a helpful assistant, please format the output as a json as {"plot types":[]}, where each element of the list refers to the plot type of a single panel. If there is a single plot, this should be one element in this list, and if there are multiple plots the list should be in row-major (C-style) order. Please choose each plot type from the following list: [line, scatter, histogram].
ANSWER: {'plot types': ['line', 'line']}


In [34]:
q_plot_types(datas, qa_pairs, plot_types_qa, return_qa=False, use_list=False)

QUESTION: What are the plot types for each panel in the figure? You are a helpful assistant, please format the output as a json as {"plot types":[]}, where each element of the list refers to the plot type of a single panel. If there is a single plot, this should be one element in this list, and if there are multiple plots the list should be in row-major (C-style) order. 
ANSWER: {'plot types': ['line', 'line']}


#### Line plot - specific questions
If line plot: how many lines?

In [35]:
# plot index to words
n_to_word = {0:'first', 1:'second', 2:'third', 3:'fourth', 4:'fifth', 5:'sixth', 6:'seventh', 7:'eigth', 8:'ninth', 9:'tenth'}
# ... for the plot in the [] panel...
def plot_index_to_words(pind):
    y = pind[0] # row
    x = pind[1] # column
    if y == 0 and x == 0:
        p = 'top-left' 
    elif y == 0:
        p = 'top row and ' + n_to_word[x] + ' from the left'
    elif x == 0:
        p = n_to_word[y] + ' row and left-most'
    else:
        p = n_to_word[x] + 'row and ' + n_to_word[y] + ' column'
    return p

In [36]:
# this version tries to give column and row numbers
def q_nlines_plot_plotnums(data, qa_pairs, plot_num = 0, return_qa=True, verbose=True):
    # rows columns
    nrow = data['figure']['plot indexes'][plot_num][0]
    ncol = data['figure']['plot indexes'][plot_num][1]
    q = 'The following question refers to the figure panel on row number ' + str(nrow) + ' and column number ' + str(ncol) + '. '
    q += 'If there are multiple plots the panels will be in row-major (C-style) order, with the numbering starting at (0,0) in the upper left panel. '
    q += 'If there is one plot, then this row and column refers to the single plot. '
    q += 'How many different lines are there for the figure panel on row number ' + str(nrow) + ' and column number ' + str(ncol) + '? '
    q += 'You are a helpful assistant, please format the output as a json as {"nlines":""} for this figure panel, where the "nlines" value should be an integer.'
    # get answer list
    la = []
    for k,v in data.items():
        if 'plot' + str(plot_num) in k: # is a plot
            #nlines = len(v['data']['data params'].keys())
            nlines = len(v['data']['ys'])
            la.append(nlines)
    a = {"nlines (plot numbers)":{'plot'+str(plot_num):la[0]}} # the 0 is a bit hacky
    if verbose:
        print('QUESTION:', q)
        print('ANSWER:', a)
    if return_qa: 
        if 'nlines (plot numbers)' not in qa_pairs['Level 1']['Plot-level questions']:
            qa_pairs['Level 1']['Plot-level questions']['nlines (plot numbers)'] = {'plot'+str(plot_num):{'Q':q, 'A':a}}
        else:
            qa_pairs['Level 1']['Plot-level questions']['nlines (plot numbers)']['plot'+str(plot_num)] = {'Q':q, 'A':a}
        return qa_pairs

In [37]:
plot_num = 0
#print(datas['figure']['plot indexes'][plot_num])

if datas['plot'+str(plot_num)]['type'] == 'line':
    q_nlines_plot_plotnums(datas, qa_pairs, plot_num=plot_num, return_qa=False)

QUESTION: The following question refers to the figure panel on row number 0 and column number 0. If there are multiple plots the panels will be in row-major (C-style) order, with the numbering starting at (0,0) in the upper left panel. If there is one plot, then this row and column refers to the single plot. How many different lines are there for the figure panel on row number 0 and column number 0? You are a helpful assistant, please format the output as a json as {"nlines":""} for this figure panel, where the "nlines" value should be an integer.
ANSWER: {'nlines (plot numbers)': {'plot0': 5}}


In [38]:
# this version tries to use words
def q_nlines_plot(data, qa_pairs, plot_num = 0, return_qa=True, verbose=True):
    # how many plots
    nplots = 0
    for k,v in data.items():
        if 'plot' in k:
            nplots += 1
    if nplots == 1: # single plot
        q = 'How many different lines are there on this figure? '
        q += 'You are a helpful assistant, please format the output as a json as {"nlines":""} for this figure panel, where the "nlines" value should be an integer.'
    else:
        q = 'How many different lines are there for the plot in the ' + plot_index_to_words(data['figure']['plot indexes'][plot_num]) + ' panel? '     
        q += 'You are a helpful assistant, please format the output as a json as {"nlines":""} for this figure panel, where the "nlines" value should be an integer.'
    # get answer list
    la = []
    for k,v in data.items():
        if 'plot' + str(plot_num) in k: # is a plot
            #nlines = len(v['data']['data params'].keys())
            nlines = len(v['data']['ys'])
            la.append(nlines)
    a = {"nlines (words)":{'plot'+str(plot_num):la[0]}} # the 0 is a bit hacky
    if verbose:
        print('QUESTION:', q)
        print('ANSWER:', a)
    if return_qa: 
        if 'nlines (words)' not in qa_pairs['Level 1']['Plot-level questions']:
            qa_pairs['Level 1']['Plot-level questions']['nlines (words)'] = {'plot'+str(plot_num):{'Q':q, 'A':a}}
        else:
            qa_pairs['Level 1']['Plot-level questions']['nlines (words)']['plot'+str(plot_num)] = {'Q':q, 'A':a}
        return qa_pairs


In [39]:
plot_num = 0
#print(datas['figure']['plot indexes'][plot_num])

if datas['plot'+str(plot_num)]['type'] == 'line':
    q_nlines_plot(datas, qa_pairs, plot_num=plot_num, return_qa=False)

QUESTION: How many different lines are there for the plot in the top-left panel? You are a helpful assistant, please format the output as a json as {"nlines":""} for this figure panel, where the "nlines" value should be an integer.
ANSWER: {'nlines (words)': {'plot0': 5}}


Colors of each line:

In [40]:
def q_colors_lines(data, qa_pairs, plot_num = 0, return_qa=True, use_words=True, verbose=True):
    """
    use_words : set to True to translate row, column to words; False will use C-ordering indexing
    """
    big_tag = 'line colors'
    if not use_words:
        adder = '(plot numbers)'
        # rows columns
        nrow = data['figure']['plot indexes'][plot_num][0]
        ncol = data['figure']['plot indexes'][plot_num][1]
        q = 'The following question refers to the figure panel on row number ' + str(nrow) + ' and column number ' + str(ncol) + '. '
        q += 'If there are multiple plots the panels will be in row-major (C-style) order, with the numbering starting at (0,0) in the upper left panel. '
        q += 'If there is one plot, then this row and column refers to the single plot. '
        q += 'What are the colors of the lines for the figure panel on row number ' + str(nrow) + ' and column number ' + str(ncol) + '? '
    else: 
        adder = '(words)'
        # how many plots
        nplots = 0
        for k,v in data.items():
            if 'plot' in k:
                nplots += 1
        if nplots == 1: # single plot
            q = 'What are the colors of the lines in this figure? '
        else:
            q = 'What are the colors of the lines for the plot in the ' + plot_index_to_words(data['figure']['plot indexes'][plot_num]) + ' panel? '     

    q += 'You are a helpful assistant, please format the output as a json as {"line colors":[]}, where each element of the list refers to the '
    q += ' color of each line in the figure panel in the form of an RGBA list with values between 0 and 1. '
    # get answer list
    la = []
    for k,v in data.items():
        if 'plot' + str(plot_num) in k: # is a plot
            for c in v['data from plot']['plot params']['colors']:
                la.append(c[0]) # the "0" here is because, in theory, the line could be made up of more than one color

    a = {big_tag + ' ' + adder:{'plot'+str(plot_num):la}} 
    if verbose:
        print('QUESTION:', q)
        print('ANSWER:', a)
    if return_qa: 
        if big_tag + ' ' + adder not in qa_pairs['Level 1']['Plot-level questions']:
            qa_pairs['Level 1']['Plot-level questions'][big_tag + ' ' + adder] = {'plot'+str(plot_num):{'Q':q, 'A':a}}
        else:
            qa_pairs['Level 1']['Plot-level questions'][big_tag + ' ' + adder]['plot'+str(plot_num)] = {'Q':q, 'A':a}
        return qa_pairs

In [41]:
plot_num = 0
print(datas['figure']['plot indexes'][plot_num])
print(plot_index_to_words(datas['figure']['plot indexes'][plot_num]))
print('')

if datas['plot'+str(plot_num)]['type'] == 'line':
    q_colors_lines(datas, qa_pairs, plot_num=plot_num, return_qa=False, use_words=False)
    print('')
    q_colors_lines(datas, qa_pairs, plot_num=plot_num, return_qa=False, use_words=True)

[0 0]
top-left

QUESTION: The following question refers to the figure panel on row number 0 and column number 0. If there are multiple plots the panels will be in row-major (C-style) order, with the numbering starting at (0,0) in the upper left panel. If there is one plot, then this row and column refers to the single plot. What are the colors of the lines for the figure panel on row number 0 and column number 0? You are a helpful assistant, please format the output as a json as {"line colors":[]}, where each element of the list refers to the  color of each line in the figure panel in the form of an RGBA list with values between 0 and 1. 
ANSWER: {'line colors (plot numbers)': {'plot0': [[0.0, 0.24705882352941178, 1.0, 1.0], [0.011764705882352941, 0.9294117647058824, 0.22745098039215686, 1.0], [0.9098039215686274, 0.0, 0.043137254901960784, 1.0], [0.5411764705882353, 0.16862745098039217, 0.8862745098039215, 1.0], [1.0, 0.7686274509803922, 0.0, 1.0]]}}

QUESTION: What are the colors of 

Linestyles of each plot:

In [42]:
def q_linestyles_lines(data, qa_pairs, plot_num = 0, return_qa=True, use_words=True, use_list=False,
                      linestyle_list = [], verbose=True):
    """
    use_words : set to True to translate row, column to words; False will use C-ordering indexing
    use_list : use a list of linestyle options for prompting
    """
    
    big_tag = 'line styles'
    if not use_words:
        adder = '(plot numbers)'
        # rows columns
        nrow = data['figure']['plot indexes'][plot_num][0]
        ncol = data['figure']['plot indexes'][plot_num][1]
        q = 'The following question refers to the figure panel on row number ' + str(nrow) + ' and column number ' + str(ncol) + '. '
        q += 'If there are multiple plots the panels will be in row-major (C-style) order, with the numbering starting at (0,0) in the upper left panel. '
        q += 'If there is one plot, then this row and column refers to the single plot. '
        q += 'What are the '+big_tag+' of the lines for the figure panel on row number ' + str(nrow) + ' and column number ' + str(ncol) + '? '
    else: 
        adder = '(words)'
        # how many plots
        nplots = 0
        for k,v in data.items():
            if 'plot' in k:
                nplots += 1
        if nplots == 1: # single plot
            q = 'What are the matplotlib linestyles in this figure? '
        else:
            q = 'What are the matplotlib linestyles for the plot in the ' + plot_index_to_words(data['figure']['plot indexes'][plot_num]) + ' panel? '     

    q += 'You are a helpful assistant, please format the output as a json as {"line styles":[]}, where each element of the list refers to the '
    q += ' linestyle of each line in the figure panel in the form a matplotlib linestyle type. '
    if use_list:
        adder = adder.split(')')[0] + ' + list)'
        q += 'Please choose each linestyle from the following list: ['
        for pt in linestyle_list:
            q += pt + ', '
        q = q[:-2] # take off the last bit
        q += '].'

    # get answer list
    la = []
    for k,v in data.items():
        if 'plot' + str(plot_num) in k: # is a plot
            for c in v['data from plot']['plot params']['linestyles']:
                la.append(c) 

    a = {big_tag + ' ' + adder:{'plot'+str(plot_num):la}} 
    if verbose:
        print('QUESTION:', q)
        print('ANSWER:', a)
    if return_qa: 
        if big_tag + ' ' + adder not in qa_pairs['Level 1']['Plot-level questions']:
            qa_pairs['Level 1']['Plot-level questions'][big_tag + ' ' + adder] = {'plot'+str(plot_num):{'Q':q, 'A':a}}
        else:
            qa_pairs['Level 1']['Plot-level questions'][big_tag + ' ' + adder]['plot'+str(plot_num)] = {'Q':q, 'A':a}
        return qa_pairs

In [43]:
plot_num = 0
print(datas['figure']['plot indexes'][plot_num])
print(plot_index_to_words(datas['figure']['plot indexes'][plot_num]))
print('')

if datas['plot'+str(plot_num)]['type'] == 'line':
    q_linestyles_lines(datas, qa_pairs, plot_num=plot_num, return_qa=False, use_words=False, use_list=False)
    print('')
    q_linestyles_lines(datas, qa_pairs, plot_num=plot_num, return_qa=False, use_words=True, use_list = False)
    print('')
    q_linestyles_lines(datas, qa_pairs, plot_num=plot_num, return_qa=False, use_words=False, use_list=True,
                      linestyle_list=linestyles)
    print('')
    q_linestyles_lines(datas, qa_pairs, plot_num=plot_num, return_qa=False, use_words=True, use_list = True,
                      linestyle_list=linestyles)


[0 0]
top-left

QUESTION: The following question refers to the figure panel on row number 0 and column number 0. If there are multiple plots the panels will be in row-major (C-style) order, with the numbering starting at (0,0) in the upper left panel. If there is one plot, then this row and column refers to the single plot. What are the line styles of the lines for the figure panel on row number 0 and column number 0? You are a helpful assistant, please format the output as a json as {"line styles":[]}, where each element of the list refers to the  linestyle of each line in the figure panel in the form a matplotlib linestyle type. 
ANSWER: {'line styles (plot numbers)': {'plot0': ['--', ':', '-', ':', '-']}}

QUESTION: What are the matplotlib linestyles for the plot in the top-left panel? You are a helpful assistant, please format the output as a json as {"line styles":[]}, where each element of the list refers to the  linestyle of each line in the figure panel in the form a matplotlib

Line thickness:

In [44]:
def q_linethickness_lines(data, qa_pairs, plot_num = 0, return_qa=True, use_words=True, use_list=False,
                      line_list = [], verbose=True):
    """
    use_words : set to True to translate row, column to words; False will use C-ordering indexing
    use_list : use a list of "property" options for prompting
    """
    
    big_tag = 'line thicknesses'
    big_tag_single = 'line thickness'
    big_tag_short = 'thickness'
    plot_param_tag = 'linethick'
    if not use_words:
        adder = '(plot numbers)'
        # rows columns
        nrow = data['figure']['plot indexes'][plot_num][0]
        ncol = data['figure']['plot indexes'][plot_num][1]
        q = 'The following question refers to the figure panel on row number ' + str(nrow) + ' and column number ' + str(ncol) + '. '
        q += 'If there are multiple plots the panels will be in row-major (C-style) order, with the numbering starting at (0,0) in the upper left panel. '
        q += 'If there is one plot, then this row and column refers to the single plot. '
        q += 'What are the '+big_tag+' of the lines for the figure panel on row number ' + str(nrow) + ' and column number ' + str(ncol) + '? '
    else: 
        adder = '(words)'
        # how many plots
        nplots = 0
        for k,v in data.items():
            if 'plot' in k:
                nplots += 1
        if nplots == 1: # single plot
            q = 'What are the matplotlib '+big_tag+' in this figure? '
        else:
            q = 'What are the matplotlib '+big_tag+' for the plot in the ' + plot_index_to_words(data['figure']['plot indexes'][plot_num]) + ' panel? '     

    q += 'You are a helpful assistant, please format the output as a json as {"'+big_tag_single+'":[]}, where each element of the list refers to the '
    q += ' '+big_tag_short+' of each line in the figure panel in the form a matplotlib '+big_tag_short+' type. '
    if use_list:
        adder = adder.split(')')[0] + ' + list)'
        q += 'Please choose each '+big_tag_short+' from the following list: ['
        for pt in line_list:
            q += pt + ', '
        q = q[:-2] # take off the last bit
        q += '].'

    # get answer list
    la = []
    for k,v in data.items():
        if 'plot' + str(plot_num) in k: # is a plot
            for c in v['data from plot']['plot params'][plot_param_tag]:
                la.append(c) 

    a = {big_tag + ' ' + adder:{'plot'+str(plot_num):la}} 
    if verbose:
        print('QUESTION:', q)
        print('ANSWER:', a)
    if return_qa: 
        if big_tag + ' ' + adder not in qa_pairs['Level 1']['Plot-level questions']:
            qa_pairs['Level 1']['Plot-level questions'][big_tag + ' ' + adder] = {'plot'+str(plot_num):{'Q':q, 'A':a}}
        else:
            qa_pairs['Level 1']['Plot-level questions'][big_tag + ' ' + adder]['plot'+str(plot_num)] = {'Q':q, 'A':a}
        return qa_pairs

In [45]:
plot_num=0
if datas['plot'+str(plot_num)]['type'] == 'line':
    q_linethickness_lines(datas, qa_pairs, plot_num=plot_num, return_qa=False, use_words=False)
    print('')
    q_linethickness_lines(datas, qa_pairs, plot_num=plot_num, return_qa=False, use_words=True)

QUESTION: The following question refers to the figure panel on row number 0 and column number 0. If there are multiple plots the panels will be in row-major (C-style) order, with the numbering starting at (0,0) in the upper left panel. If there is one plot, then this row and column refers to the single plot. What are the line thicknesses of the lines for the figure panel on row number 0 and column number 0? You are a helpful assistant, please format the output as a json as {"line thickness":[]}, where each element of the list refers to the  thickness of each line in the figure panel in the form a matplotlib thickness type. 
ANSWER: {'line thicknesses (plot numbers)': {'plot0': [2.192727398337825, 2.430835373225659, 2.09361447179014, 3.9596997525227, 4.38076267304099]}}

QUESTION: What are the matplotlib line thicknesses for the plot in the top-left panel? You are a helpful assistant, please format the output as a json as {"line thickness":[]}, where each element of the list refers to t

Line markers:

In [46]:
def q_linemarkers_lines(data, qa_pairs, plot_num = 0, return_qa=True, use_words=True, use_list=False,
                      line_list = [], verbose=True):
    """
    use_words : set to True to translate row, column to words; False will use C-ordering indexing
    use_list : use a list of "property" options for prompting
    """
    
    big_tag = 'line markers'
    big_tag_single = 'line marker'
    big_tag_short = 'marker'
    plot_param_tag = 'markers'
    extra_if_empty = 'If there are no markers for a specific line, please indicate this with an empty "" for this element of the list.'
    if not use_words:
        adder = '(plot numbers)'
        # rows columns
        nrow = data['figure']['plot indexes'][plot_num][0]
        ncol = data['figure']['plot indexes'][plot_num][1]
        q = 'The following question refers to the figure panel on row number ' + str(nrow) + ' and column number ' + str(ncol) + '. '
        q += 'If there are multiple plots the panels will be in row-major (C-style) order, with the numbering starting at (0,0) in the upper left panel. '
        q += 'If there is one plot, then this row and column refers to the single plot. '
        q += 'What are the '+big_tag+' of the lines for the figure panel on row number ' + str(nrow) + ' and column number ' + str(ncol) + '? '
    else: 
        adder = '(words)'
        # how many plots
        nplots = 0
        for k,v in data.items():
            if 'plot' in k:
                nplots += 1
        if nplots == 1: # single plot
            q = 'What are the matplotlib '+big_tag+' in this figure? '
        else:
            q = 'What are the matplotlib '+big_tag+' for the plot in the ' + plot_index_to_words(data['figure']['plot indexes'][plot_num]) + ' panel? '     

    q += 'You are a helpful assistant, please format the output as a json as {"'+big_tag_single+'":[]}, where each element of the list refers to the '
    q += ' '+big_tag_short+' of each line in the figure panel in the form a matplotlib '+big_tag_short+' type. '
    q += extra_if_empty
    if use_list:
        adder = adder.split(')')[0] + ' + list)'
        q += 'Please choose each '+big_tag_short+' from the following list: ['
        for pt in line_list:
            q += pt + ', '
        q = q[:-2] # take off the last bit
        q += '].'

    # get answer list
    la = []
    for k,v in data.items():
        if 'plot' + str(plot_num) in k: # is a plot
            for c in v['data from plot']['plot params'][plot_param_tag]:
                la.append(c) 

    a = {big_tag + ' ' + adder:{'plot'+str(plot_num):la}} 
    if verbose:
        print('QUESTION:', q)
        print('ANSWER:', a)
    if return_qa: 
        if big_tag + ' ' + adder not in qa_pairs['Level 1']['Plot-level questions']:
            qa_pairs['Level 1']['Plot-level questions'][big_tag + ' ' + adder] = {'plot'+str(plot_num):{'Q':q, 'A':a}}
        else:
            qa_pairs['Level 1']['Plot-level questions'][big_tag + ' ' + adder]['plot'+str(plot_num)] = {'Q':q, 'A':a}
        return qa_pairs

In [47]:
plot_num=0
if datas['plot'+str(plot_num)]['type'] == 'line':
    q_linemarkers_lines(datas, qa_pairs, plot_num=plot_num, return_qa=False, use_words=False)
    print('')
    q_linemarkers_lines(datas, qa_pairs, plot_num=plot_num, return_qa=False, use_words=True)

QUESTION: The following question refers to the figure panel on row number 0 and column number 0. If there are multiple plots the panels will be in row-major (C-style) order, with the numbering starting at (0,0) in the upper left panel. If there is one plot, then this row and column refers to the single plot. What are the line markers of the lines for the figure panel on row number 0 and column number 0? You are a helpful assistant, please format the output as a json as {"line marker":[]}, where each element of the list refers to the  marker of each line in the figure panel in the form a matplotlib marker type. If there are no markers for a specific line, please indicate this with an empty "" for this element of the list.
ANSWER: {'line markers (plot numbers)': {'plot0': ['', '', '', '', '']}}

QUESTION: What are the matplotlib line markers for the plot in the top-left panel? You are a helpful assistant, please format the output as a json as {"line marker":[]}, where each element of the

In [48]:
def q_linemarkersize_lines(data, qa_pairs, plot_num = 0, return_qa=True, use_words=True, use_list=False,
                      line_list = [], verbose=True):
    """
    use_words : set to True to translate row, column to words; False will use C-ordering indexing
    use_list : use a list of "property" options for prompting
    """
    
    big_tag = 'line marker sizes'
    big_tag_single = 'line marker size'
    big_tag_short = 'marker size'
    plot_param_tag = 'marker size' # tag in the plot params
    extra_if_empty = 'If there are no markers for a specific line, please indicate this with an empty "" for this element of the list.'
    if not use_words:
        adder = '(plot numbers)'
        # rows columns
        nrow = data['figure']['plot indexes'][plot_num][0]
        ncol = data['figure']['plot indexes'][plot_num][1]
        q = 'The following question refers to the figure panel on row number ' + str(nrow) + ' and column number ' + str(ncol) + '. '
        q += 'If there are multiple plots the panels will be in row-major (C-style) order, with the numbering starting at (0,0) in the upper left panel. '
        q += 'If there is one plot, then this row and column refers to the single plot. '
        q += 'What are the '+big_tag+' of the lines for the figure panel on row number ' + str(nrow) + ' and column number ' + str(ncol) + '? '
    else: 
        adder = '(words)'
        # how many plots
        nplots = 0
        for k,v in data.items():
            if 'plot' in k:
                nplots += 1
        if nplots == 1: # single plot
            q = 'What are the matplotlib '+big_tag+' in this figure? '
        else:
            q = 'What are the matplotlib '+big_tag+' for the plot in the ' + plot_index_to_words(data['figure']['plot indexes'][plot_num]) + ' panel? '     

    q += 'You are a helpful assistant, please format the output as a json as {"'+big_tag_single+'":[]}, where each element of the list refers to the '
    q += ' '+big_tag_short+' of each line in the figure panel in the form a matplotlib '+big_tag_short+' type. '
    q += extra_if_empty
    if use_list:
        adder = adder.split(')')[0] + ' + list)'
        q += 'Please choose each '+big_tag_short+' from the following list: ['
        for pt in line_list:
            q += pt + ', '
        q = q[:-2] # take off the last bit
        q += '].'

    # get answer list
    la = []
    for k,v in data.items():
        if 'plot' + str(plot_num) in k: # is a plot
            for c in v['data from plot']['plot params'][plot_param_tag]:
                la.append(c) 

    a = {big_tag + ' ' + adder:{'plot'+str(plot_num):la}} 
    if verbose:
        print('QUESTION:', q)
        print('ANSWER:', a)
    if return_qa: 
        if big_tag + ' ' + adder not in qa_pairs['Level 1']['Plot-level questions']:
            qa_pairs['Level 1']['Plot-level questions'][big_tag + ' ' + adder] = {'plot'+str(plot_num):{'Q':q, 'A':a}}
        else:
            qa_pairs['Level 1']['Plot-level questions'][big_tag + ' ' + adder]['plot'+str(plot_num)] = {'Q':q, 'A':a}
        return qa_pairs

In [49]:
plot_num=0
if datas['plot'+str(plot_num)]['type'] == 'line':
    q_linemarkersize_lines(datas, qa_pairs, plot_num=plot_num, return_qa=False, use_words=False)
    print('')
    q_linemarkersize_lines(datas, qa_pairs, plot_num=plot_num, return_qa=False, use_words=True)

QUESTION: The following question refers to the figure panel on row number 0 and column number 0. If there are multiple plots the panels will be in row-major (C-style) order, with the numbering starting at (0,0) in the upper left panel. If there is one plot, then this row and column refers to the single plot. What are the line marker sizes of the lines for the figure panel on row number 0 and column number 0? You are a helpful assistant, please format the output as a json as {"line marker size":[]}, where each element of the list refers to the  marker size of each line in the figure panel in the form a matplotlib marker size type. If there are no markers for a specific line, please indicate this with an empty "" for this element of the list.
ANSWER: {'line marker sizes (plot numbers)': {'plot0': [-1, -1, -1, -1, -1]}}

QUESTION: What are the matplotlib line marker sizes for the plot in the top-left panel? You are a helpful assistant, please format the output as a json as {"line marker s

Values of x & y data points:

In [50]:
def q_datapoints_lines(data, qa_pairs, plot_num = 0, return_qa=True, use_words=True, 
                      line_list = [], verbose=True):
    """
    use_words : set to True to translate row, column to words; False will use C-ordering indexing
    use_list : use a list of "property" options for prompting
    """
    big_tag = 'data values'
    # how many plots
    nplots = 0
    for k,v in data.items(): # count number of plots
        if 'plot' in k:
            nplots += 1
    if not use_words:
        adder = '(plot numbers)'
        # rows columns
        nrow = data['figure']['plot indexes'][plot_num][0]
        ncol = data['figure']['plot indexes'][plot_num][1]
        q = 'The following question refers to the figure panel on row number ' + str(nrow) + ' and column number ' + str(ncol) + '. '
        q += 'If there are multiple plots the panels will be in row-major (C-style) order, with the numbering starting at (0,0) in the upper left panel. '
        q += 'If there is one plot, then this row and column refers to the single plot. '
    else: 
        adder = '(words)'

    if nplots == 1: # single plot
        q = 'What are the data values in this figure? '
    else:
        if not use_words:
            q += 'What are the data values for the figure panel on row number ' + str(nrow) + ' and column number ' + str(ncol) + '? '
        else:
            q = 'What are the data values for the plot in the ' + plot_index_to_words(data['figure']['plot indexes'][plot_num]) + ' panel? '     

    q += 'You are a helpful assistant, please format the output as a json as {"x":[], "y":[]} where the values of "x" and "y" are the '
    q += 'data values used to create the plot.  '
    q += 'If there is one single array for x data values for all lines on the plot, "x" will be a single list, otherwise it will be a list of lists. '
    q += 'If there is one single array for y data values for all lines on the plot, "y" will be a single list, otherwise it will be a list of lists. '
    
    # get answer list
    la = {"x":data['plot'+str(plot_num)]['data']['xs'], "y":data['plot'+str(plot_num)]['data']['ys']}
    a = {big_tag + ' ' + adder:{'plot'+str(plot_num):la}} 
    if verbose:
        print('QUESTION:', q)
        print('ANSWER:', a)
    if return_qa: 
        if big_tag + ' ' + adder not in qa_pairs['Level 1']['Plot-level questions']:
            qa_pairs['Level 1']['Plot-level questions'][big_tag + ' ' + adder] = {'plot'+str(plot_num):{'Q':q, 'A':a}}
        else:
            qa_pairs['Level 1']['Plot-level questions'][big_tag + ' ' + adder]['plot'+str(plot_num)] = {'Q':q, 'A':a}
        return qa_pairs

In [51]:
plot_num=0
if datas['plot'+str(plot_num)]['type'] == 'line':
    q_datapoints_lines(datas, qa_pairs, plot_num=plot_num, return_qa=False, use_words=False)
    print('')
    q_datapoints_lines(datas, qa_pairs, plot_num=plot_num, return_qa=False, use_words=True)

QUESTION: The following question refers to the figure panel on row number 0 and column number 0. If there are multiple plots the panels will be in row-major (C-style) order, with the numbering starting at (0,0) in the upper left panel. If there is one plot, then this row and column refers to the single plot. What are the data values for the figure panel on row number 0 and column number 0? You are a helpful assistant, please format the output as a json as {"x":[], "y":[]} where the values of "x" and "y" are the data values used to create the plot.  If there is one single array for x data values for all lines on the plot, "x" will be a single list, otherwise it will be a list of lists. If there is one single array for y data values for all lines on the plot, "y" will be a single list, otherwise it will be a list of lists. 
ANSWER: {'data values (plot numbers)': {'plot0': {'x': [array([-2707.15538097, -2599.925314  , -2510.87159992, -2482.63455567,
       -2298.18891112, -2283.80258526, 

### Higher-level: L2

#### Plot-level questions
1. Stats for each plot

In [52]:
stats = {'minimum':np.min, 'maximum':np.max, 'median':np.median, 'mean':np.mean}
#stats.keys()
{'minimum':np.min}.values()

dict_values([<function min at 0x10b938d70>])

In [53]:
def q_stats_lines(data, qa_pairs, stat = {'minimum':np.min}, plot_num = 0, return_qa=True, use_words=True, verbose=True):
    """
    use_words : set to True to translate row, column to words; False will use C-ordering indexing
    stat : dictionary of the name and function to use for each stat
    """
    big_tag = list(stat.keys())[0]
    # how many plots
    nplots = 0
    for k,v in data.items(): # count number of plots
        if 'plot' in k:
            nplots += 1
    if not use_words:
        adder = '(plot numbers)'
        # rows columns
        nrow = data['figure']['plot indexes'][plot_num][0]
        ncol = data['figure']['plot indexes'][plot_num][1]
        q = 'The following question refers to the figure panel on row number ' + str(nrow) + ' and column number ' + str(ncol) + '. '
        q += 'If there are multiple plots the panels will be in row-major (C-style) order, with the numbering starting at (0,0) in the upper left panel. '
        q += 'If there is one plot, then this row and column refers to the single plot. '
    else: 
        adder = '(words)'

    if nplots == 1: # single plot
        q = 'What are the '+big_tag+' data values in this figure? '
    else:
        if not use_words:
            q += 'What are the '+big_tag+' data values for the figure panel on row number ' + str(nrow) + ' and column number ' + str(ncol) + '? '
        else:
            q = 'What are the '+big_tag+' data values for the plot in the ' + plot_index_to_words(data['figure']['plot indexes'][plot_num]) + ' panel? '     

    q += 'You are a helpful assistant, please format the output as a json as {"'+big_tag+' x":[], "'+big_tag+' y":[]} where the '+big_tag+' values of "x" and "y" are the '
    q += 'data values used to create the plot.  '
    q += 'If there is one single array for x data values for all lines on the plot, "'+big_tag+' x" will be a single item list. '
    q += 'If there is one single array for y data values for all lines on the plot, "'+big_tag+' y" will be a single item list. '
    #q += 'If there is one single array for y data values for all lines on the plot, "minimum y" will be a single list, otherwise it will be a list of lists. '
    
    # get answer list
    f = list(stat.values())[0]
    xs = []; ys = []
    for x in data['plot'+str(plot_num)]['data']['xs']:
        xs.append(f(x))
    for y in data['plot'+str(plot_num)]['data']['ys']:
        ys.append(f(y))
    la = {big_tag + " x":xs, big_tag + " y":ys}
    a = {big_tag + ' ' + adder:{'plot'+str(plot_num):la}} 
    if verbose:
        print('QUESTION:', q)
        print('ANSWER:', a)
    #a = list(aout.values())[0]
    if return_qa: 
        if big_tag + ' ' + adder not in qa_pairs['Level 2']['Plot-level questions']:
            qa_pairs['Level 2']['Plot-level questions'][big_tag + ' ' + adder] = {'plot'+str(plot_num):{'Q':q, 'A':a}}
        else:
            qa_pairs['Level 2']['Plot-level questions'][big_tag + ' ' + adder]['plot'+str(plot_num)] = {'Q':q, 'A':a}
        return qa_pairs

In [54]:
plot_num=0
if datas['plot'+str(plot_num)]['type'] == 'line':
    for k,v in stats.items():
        q_stats_lines(datas, qa_pairs, stat={k:v}, plot_num=plot_num, return_qa=False, use_words=False)
        print('')
        q_stats_lines(datas, qa_pairs, stat={k:v}, plot_num=plot_num, return_qa=False, use_words=True)
        print('')
        print('')

QUESTION: The following question refers to the figure panel on row number 0 and column number 0. If there are multiple plots the panels will be in row-major (C-style) order, with the numbering starting at (0,0) in the upper left panel. If there is one plot, then this row and column refers to the single plot. What are the minimum data values for the figure panel on row number 0 and column number 0? You are a helpful assistant, please format the output as a json as {"minimum x":[], "minimum y":[]} where the minimum values of "x" and "y" are the data values used to create the plot.  If there is one single array for x data values for all lines on the plot, "minimum x" will be a single item list. If there is one single array for y data values for all lines on the plot, "minimum y" will be a single item list. 
ANSWER: {'minimum (plot numbers)': {'plot0': {'minimum x': [-2707.155380971784, -2693.8595331204797, -2635.283539008062, -2695.539566860079, -2677.396385290205], 'minimum y': [-3.71266

In [55]:
def q_errorbars_existance_lines(data, qa_pairs, axis = 'x', plot_num = 0, return_qa=True, use_words=True, verbose=True):
    """
    use_words : set to True to translate row, column to words; False will use C-ordering indexing
    """
    
    # how many plots
    nplots = 0
    for k,v in data.items(): # count number of plots
        if 'plot' in k:
            nplots += 1
    if not use_words:
        adder = '(plot numbers)'
        # rows columns
        nrow = data['figure']['plot indexes'][plot_num][0]
        ncol = data['figure']['plot indexes'][plot_num][1]
        q = 'The following question refers to the figure panel on row number ' + str(nrow) + ' and column number ' + str(ncol) + '. '
        q += 'If there are multiple plots the panels will be in row-major (C-style) order, with the numbering starting at (0,0) in the upper left panel. '
        q += 'If there is one plot, then this row and column refers to the single plot. '
    else: 
        adder = '(words)'

    if nplots == 1: # single plot
        q = 'Are there error bars on the data along the ' +axis+ '-axis in this figure? '
    else:
        if not use_words:
            q += 'Are there error bars on the data along the ' +axis+ '-axis for the figure panel on row number ' + str(nrow) + ' and column number ' + str(ncol) + '? '
        else:
            q = 'Are there error bars on the data along the ' +axis+ '-axis for the plot in the ' + plot_index_to_words(data['figure']['plot indexes'][plot_num]) + ' panel? '     

    # q += 'You are a helpful assistant, please format the output as a json as {"'+axis+'-axis errors":[]} where the value for output is True '
    # q += '(if error bars exist) or False (if error bars do not exist). '
    # q += 'If there is one single array for '+axis+' data values for all lines on the plot, there will be a single True or False in the list, otherwise it will be list of True/False. '
    q += 'You are a helpful assistant, please format the output as a json as {"'+axis+'-axis errors":""} where the value for the output is True '
    q += '(if error bars exist) or False (if error bars do not exist). '

    # get answer
    if axis+'errs' in data['plot' + str(plot_num)]['data'].keys():
        a = {axis+'-axis errors' + ' ' + adder:True}
    else:
        a = {axis+'-axis errors' + ' ' + adder:False}

    #a = list(aout.values())[0]
    if verbose:
        print('QUESTION:', q)
        print('ANSWER:', a)
    if return_qa: 
        if axis+'-axis errors' + ' ' + adder not in qa_pairs['Level 2']['Plot-level questions']:
            qa_pairs['Level 2']['Plot-level questions'][axis+'-axis errors' + ' ' + adder] = {'plot'+str(plot_num):{'Q':q, 'A':a}}
        else:
            qa_pairs['Level 2']['Plot-level questions'][axis+'-axis errors' + ' ' + adder]['plot'+str(plot_num)] = {'Q':q, 'A':a}
        return qa_pairs

In [56]:
iplot=0
if datas['plot'+str(plot_num)]['type'] == 'line':
    q_errorbars_existance_lines(datas, qa_pairs, axis='x', plot_num=iplot, return_qa=False, use_words=False)
    q_errorbars_existance_lines(datas, qa_pairs, axis='y', plot_num=iplot, return_qa=False, use_words=False)
    print('')
    q_errorbars_existance_lines(datas, qa_pairs, axis='x', plot_num=iplot, return_qa=False, use_words=True)
    q_errorbars_existance_lines(datas, qa_pairs, axis='y', plot_num=iplot, return_qa=False, use_words=True)

QUESTION: The following question refers to the figure panel on row number 0 and column number 0. If there are multiple plots the panels will be in row-major (C-style) order, with the numbering starting at (0,0) in the upper left panel. If there is one plot, then this row and column refers to the single plot. Are there error bars on the data along the x-axis for the figure panel on row number 0 and column number 0? You are a helpful assistant, please format the output as a json as {"x-axis errors":""} where the value for the output is True (if error bars exist) or False (if error bars do not exist). 
ANSWER: {'x-axis errors (plot numbers)': False}
QUESTION: The following question refers to the figure panel on row number 0 and column number 0. If there are multiple plots the panels will be in row-major (C-style) order, with the numbering starting at (0,0) in the upper left panel. If there is one plot, then this row and column refers to the single plot. Are there error bars on the data al

In [57]:
def q_errorbars_size_lines(data, qa_pairs, axis = 'x', plot_num = 0, return_qa=True, use_words=True, verbose=True):
    """
    use_words : set to True to translate row, column to words; False will use C-ordering indexing
    """
    
    # how many plots
    nplots = 0
    for k,v in data.items(): # count number of plots
        if 'plot' in k:
            nplots += 1
    if not use_words:
        adder = '(plot numbers)'
        # rows columns
        nrow = data['figure']['plot indexes'][plot_num][0]
        ncol = data['figure']['plot indexes'][plot_num][1]
        q = 'The following question refers to the figure panel on row number ' + str(nrow) + ' and column number ' + str(ncol) + '. '
        q += 'If there are multiple plots the panels will be in row-major (C-style) order, with the numbering starting at (0,0) in the upper left panel. '
        q += 'If there is one plot, then this row and column refers to the single plot. '
    else: 
        adder = '(words)'

    if nplots == 1: # single plot
        q = 'What is the approximate size of the error bars along the ' + axis + '-axis in this figure? '
    else:
        if not use_words:
            q += 'What is the approximate size of the error bars along the ' + axis + '-axis for the figure panel on row number ' + str(nrow) + ' and column number ' + str(ncol) + '? '
        else:
            q = 'What is the approximate size of the error bars along the ' + axis + '-axis for the plot in the ' + plot_index_to_words(data['figure']['plot indexes'][plot_num]) + ' panel? '     

    q += 'You are a helpful assistant, please format the output as a json as {"'+axis+'-axis error size":[]} where each element of the list corresponds to a single line in the figure. '
    q += 'The size of the error bar can be calculated as the average length of the bar divided by the difference between the maximum and minimum data values along the '+axis+'-axis.'

    # get answer
    if axis+'errs' in data['plot' + str(plot_num)]['data'].keys(): # just a double check
        errs = []
        for ev,v in zip(data['plot'+str(plot_num)]['data'][axis+'errs'],data['plot'+str(plot_num)]['data'][axis+'s']): # all lines
            errs.append(np.mean(np.abs(ev))/(np.max(v)-np.min(v)))
        a = {axis+'-axis errors size' + ' ' + adder:errs}
    else:
        print('No axis error bars!!')
        import sys; sys.exit()

    #a = aout.values[0]

    if verbose:
        print('QUESTION:', q)
        print('ANSWER:', a)
    if return_qa: 
        if axis+'-axis error size' + ' ' + adder not in qa_pairs['Level 2']['Plot-level questions']:
            qa_pairs['Level 2']['Plot-level questions'][axis+'-axis error size' + ' ' + adder] = {'plot'+str(plot_num):{'Q':q, 'A':a}}
        else:
            qa_pairs['Level 2']['Plot-level questions'][axis+'-axis error size' + ' ' + adder]['plot'+str(plot_num)] = {'Q':q, 'A':a}
        return qa_pairs

In [58]:
iplot=0
if datas['plot'+str(plot_num)]['type'] == 'line':
    for axis in ['x','y']:
        if axis + 'errs' in datas['plot'+str(iplot)]['data']:
            q_errorbars_size_lines(datas, qa_pairs, axis=axis, plot_num=iplot, return_qa=False, use_words=False)
            print('')
            q_errorbars_size_lines(datas, qa_pairs, axis=axis, plot_num=iplot, return_qa=False, use_words=True)

### Even Higher-level: L2/L3 types

#### Plot-level questions
Relationship for each plot

In [59]:
def q_relationship_lines(data, qa_pairs, plot_num = 0, return_qa=True, use_words=True, use_list=True, use_nlines = True,
                        line_list = ['random','linear','gaussian mixture model'], verbose=True):
    """
    use_words : set to True to translate row, column to words; False will use C-ordering indexing
    use_list : give a list of possible distributions
    use_nplots : give the number of lines in the prompt
    """
    
    # how many plots
    big_tag_short = 'relationship'
    nplots = 0
    for k,v in data.items(): # count number of plots
        if 'plot' in k:
            nplots += 1
    if not use_words:
        adder = '(plot numbers)'
        # rows columns
        nrow = data['figure']['plot indexes'][plot_num][0]
        ncol = data['figure']['plot indexes'][plot_num][1]
        q = 'The following question refers to the figure panel on row number ' + str(nrow) + ' and column number ' + str(ncol) + '. '
        q += 'If there are multiple plots the panels will be in row-major (C-style) order, with the numbering starting at (0,0) in the upper left panel. '
        q += 'If there is one plot, then this row and column refers to the single plot. '
    else: 
        adder = '(words)'

    if nplots == 1: # single plot
        q = 'What is the functional relationship between the x and y values in this figure? '
    else:
        if not use_words:
            q += 'What is the functional relationship between the x and y values for the figure panel on row number ' + str(nrow) + ' and column number ' + str(ncol) + '? '
        else:
            q = 'What is the functional relationship between the x and y values for the plot in the ' + plot_index_to_words(data['figure']['plot indexes'][plot_num]) + ' panel? '     

    q += 'You are a helpful assistant, please format the output as a json as {"relationship":[]} where each element of the list corresponds to a single line in the figure. '
    if use_nlines:
        adder = adder.split(')')[0] + ' + nlines)'
        q += 'Please note that there are a total of ' + str(int(len(data['plot' + str(plot_num)]['data']['ys']))) + ' lines in this plot, so the list should have a '
        q += 'total of ' +str(int(len(data['plot' + str(plot_num)]['data']['ys'])))+ ' entries. '

    if use_list:
        adder = adder.split(')')[0] + ' + list)'
        q += 'Please choose each '+big_tag_short+' for each line from the following list for each line: ['
        for pt in line_list:
            q += pt + ', '
        q = q[:-2] # take off the last bit
        q += '].'

    # answer
    la = []
    for i in range(len(data['plot' + str(plot_num)]['data']['ys'])):
        dist = data['plot'+str(plot_num)]['distribution']
        # to match
        if dist == 'gmm': dist = 'gaussian mixture model'
        la.append(dist) # note: assumes same for all!
    #a = la
    a = {big_tag_short + ' ' + adder:la}

    if verbose:
        print('QUESTION:', q)
        print('ANSWER:', a)
    if return_qa: 
        if big_tag_short + ' ' + adder not in qa_pairs['Level 3']['Plot-level questions']:
            #print('yes', big_tag_short + ' ' + adder)
            qa_pairs['Level 3']['Plot-level questions'][big_tag_short + ' ' + adder] = {'plot'+str(plot_num):{'Q':q, 'A':a, 
                                                                                                              'note':'this currently assumes all elements on a single plot have the same relationship type'}}
        else:
            qa_pairs['Level 3']['Plot-level questions'][big_tag_short + ' ' + adder]['plot'+str(plot_num)] = {'Q':q, 'A':a, 
                                                                                                              'note':'this currently assumes all elements on a single plot have the same relationship type'}
        return qa_pairs

In [60]:
#iplot=0

qa_pairs_tmp = {}
qa_pairs_tmp['Level 3'] = {}
qa_pairs_tmp['Level 3']['Plot-level questions'] = {}

for iplot in range(len((axes))):
    if datas['plot'+str(iplot)]['type'] == 'line':
        qa_pairs_tmp = q_relationship_lines(datas, qa_pairs_tmp, plot_num=iplot, use_words=False, use_nlines = False, verbose=False)
        #print('')
        qa_pairs_tmp = q_relationship_lines(datas, qa_pairs_tmp, plot_num=iplot, use_words=True, use_nlines = False, verbose=False)
        #print('')
        #print('')
        qa_pairs_tmp = q_relationship_lines(datas, qa_pairs_tmp, plot_num=iplot, use_words=False, use_nlines = True, verbose=False)
        #print('')
        qa_pairs_tmp = q_relationship_lines(datas, qa_pairs_tmp, plot_num=iplot, use_words=True, use_nlines = True, verbose = False)
        #import sys; sys.exit()

In [61]:
qa_pairs_tmp

{'Level 3': {'Plot-level questions': {'relationship (plot numbers + list)': {'plot0': {'Q': 'The following question refers to the figure panel on row number 0 and column number 0. If there are multiple plots the panels will be in row-major (C-style) order, with the numbering starting at (0,0) in the upper left panel. If there is one plot, then this row and column refers to the single plot. What is the functional relationship between the x and y values for the figure panel on row number 0 and column number 0? You are a helpful assistant, please format the output as a json as {"relationship":[]} where each element of the list corresponds to a single line in the figure. Please choose each relationship for each line from the following list for each line: [random, linear, gaussian mixture model].',
     'A': {'relationship (plot numbers + list)': ['gaussian mixture model',
       'gaussian mixture model',
       'gaussian mixture model',
       'gaussian mixture model',
       'gaussian mix

In [62]:
def q_linear_equation_lines(data, qa_pairs, plot_num = 0, return_qa=True, use_words=True, use_nlines = True, verbose=True):
    """
    use_words : set to True to translate row, column to words; False will use C-ordering indexing
    use_nlines : give the number of lines in the prompt
    """

    big_tag_short = 'linear parameters'
    # is this a linear relationship? if not this is an error
    hasLine = False
    #if 'data params' not in data['plot'+str(plot_num)]['data']: # not a linear relationship
    if data['plot'+str(plot_num)]['distribution'] != 'linear': # not a linear relationship
        if verbose:
            print('Not a linear relationship!')
            if return_qa:
                return qa_pairs
            else:
                return None
    #elif data['distribution'] 
                
    for k,v in data['plot'+str(plot_num)]['data']['data params'].items():
        if 'line' in k:
            hasLine = True
    if not hasLine:
        print("This is not a linear relationship, can't ask any questions!'")
        import sys; sys.exit()
    
    # how many plots
    nplots = 0
    for k,v in data.items(): # count number of plots
        if 'plot' in k:
            nplots += 1
    if not use_words:
        adder = '(plot numbers)'
        # rows columns
        nrow = data['figure']['plot indexes'][plot_num][0]
        ncol = data['figure']['plot indexes'][plot_num][1]
        q = 'The following question refers to the figure panel on row number ' + str(nrow) + ' and column number ' + str(ncol) + '. '
        q += 'If there are multiple plots the panels will be in row-major (C-style) order, with the numbering starting at (0,0) in the upper left panel. '
        q += 'If there is one plot, then this row and column refers to the single plot. '
    else: 
        adder = '(words)'

    if nplots == 1: # single plot
        q = 'What are the parameters for the linear relationship between the x and y values in this figure? '
    else:
        if not use_words:
            q += 'What are the parameters for the linear relationship between for the figure panel on row number ' + str(nrow) + ' and column number ' + str(ncol) + '? '
        else:
            q = 'What are the parameters for the linear relationship between the x and y values for the plot in the ' + plot_index_to_words(data['figure']['plot indexes'][plot_num]) + ' panel? '     

    q += 'You are a helpful assistant, please format the output as a json as {"'+big_tag_short+'":[{"m":"","a":"","noise level":""}]} '
    q += 'where each dictionary element of the list corresponds to a single line in the figure. '
    q += 'Each "m" is the slope for each line and "a" is the intercept for each line and should be a float. '
    q += 'The "noise level" parameter should be the relative amount of noise added to each linear function for each line and should be a float between 0 and 1. '
    
    if use_nlines:
        adder = adder.split(')')[0] + ' + nlines)'
        q += 'Please note that there are a total of ' + str(int(len(data['plot' + str(plot_num)]['data']['ys']))) + ' lines in this plot, so the list should have a '
        q += 'total of ' +str(int(len(data['plot' + str(plot_num)]['data']['ys'])))+ ' entries. '

    # answer
    la = []
    for k,v in data['plot'+str(plot_num)]['data']['data params'].items():
        la.append(v)

    a = {big_tag_short + ' ' + adder:la}

    if verbose:
        print('QUESTION:', q)
        print('ANSWER:', a)
    if return_qa: 
        if big_tag_short + ' ' + adder not in qa_pairs['Level 3']['Plot-level questions']:
            qa_pairs['Level 3']['Plot-level questions'][big_tag_short + ' ' + adder] = {'plot'+str(plot_num):{'Q':q, 'A':a, 
                                                                                                              'note':'this currently assumes all elements on a single plot have the same relationship type'}}
        else:
            qa_pairs['Level 3']['Plot-level questions'][big_tag_short + ' ' + adder]['plot'+str(plot_num)] = {'Q':q, 'A':a, 
                                                                                                              'note':'this currently assumes all elements on a single plot have the same relationship type'}
        return qa_pairs

In [63]:
iplot=0
if datas['plot'+str(plot_num)]['type'] == 'line':
    hasLine = False
    if 'data params' not in datas['plot'+str(iplot)]['data']:
        print('Not a linear relationship!')
    else:
        for k,v in datas['plot'+str(iplot)]['data']['data params'].items():
            if 'line' in k:
                hasLine = True
    if hasLine:
        q_linear_equation_lines(datas, qa_pairs, plot_num=iplot, return_qa=False, use_words=False, use_nlines = False)
        print('')
        q_linear_equation_lines(datas, qa_pairs, plot_num=iplot, return_qa=False, use_words=True, use_nlines = False)
        print('')
        print('')
        q_linear_equation_lines(datas, qa_pairs, plot_num=iplot, return_qa=False, use_words=False, use_nlines = True)
        print('')
        q_linear_equation_lines(datas, qa_pairs, plot_num=iplot, return_qa=False, use_words=True, use_nlines = True)

Not a linear relationship!


In [64]:
def q_linear_gmm_lines(data, qa_pairs, plot_num = 0, return_qa=True, use_words=True, use_nlines = True, verbose=True):
    """
    use_words : set to True to translate row, column to words; False will use C-ordering indexing
    use_nlines : give the number of lines in the prompt
    """

    big_tag_short = 'gmm parameters'
    # is this a linear relationship? if not this is an error
    hasLine = False
    if data['plot'+str(plot_num)]['distribution'] != 'gmm': # not a right relationship
        if verbose:
            print('Not a gmm relationship!')
            if return_qa:
                return qa_pairs
            else:
                return None
                
    for k,v in data['plot'+str(plot_num)]['data']['data params'].items():
        if 'line' in k:
            hasLine = True
    if not hasLine:
        print("This is not a gmm relationship, can't ask any questions!'")
        import sys; sys.exit()
    
    # how many plots
    nplots = 0
    for k,v in data.items(): # count number of plots
        if 'plot' in k:
            nplots += 1
    if not use_words:
        adder = '(plot numbers)'
        # rows columns
        nrow = data['figure']['plot indexes'][plot_num][0]
        ncol = data['figure']['plot indexes'][plot_num][1]
        q = 'The following question refers to the figure panel on row number ' + str(nrow) + ' and column number ' + str(ncol) + '. '
        q += 'If there are multiple plots the panels will be in row-major (C-style) order, with the numbering starting at (0,0) in the upper left panel. '
        q += 'If there is one plot, then this row and column refers to the single plot. '
    else: 
        adder = '(words)'

    if nplots == 1: # single plot
        q = 'What are the parameters for the gaussian mixture model relationship between the x and y values in this figure? '
    else:
        if not use_words:
            q += 'What are the parameters for the gaussian mixture model relationship between for the figure panel on row number ' \
               + str(nrow) + ' and column number ' + str(ncol) + '? '
        else:
            q = 'What are the parameters for the gaussian mixture model relationship between the x and y values for the plot in the ' \
               + plot_index_to_words(data['figure']['plot indexes'][plot_num]) + ' panel? '     

    # 'nsamples', 'nclusters', 'centers', 'cluster_std', 'noise level'
    q += 'You are a helpful assistant, please format the output as a json as {"'+big_tag_short+\
      '":[{"nsamples":"","nclusters":"","centers":"","cluster_std":"","noise level":""}]} '
    q += 'where each dictionary element of the list corresponds to a single line in the figure. '
    q += 'Each "nsamples" is the number of samples in the distribution and should be an integer. '
    q += 'The "nclusters" parameter should be the number of clusters in the model and should be an integer. '
    q += 'The "centers" parameter should be the position of each cluster on the x-axis and should be a float. '
    q += 'The "cluster_std" parameter should be the standard deviation for each cluster on the x-axis and should be a float. '
    q += 'The "noise level" parameter should be the relative amount of noise added to each linear function for each line and should be a float between 0 and 1. '
    
    if use_nlines:
        adder = adder.split(')')[0] + ' + nlines)'
        q += 'Please note that there are a total of ' + str(int(len(data['plot' + str(plot_num)]['data']['ys']))) + ' lines in this plot, so the list should have a '
        q += 'total of ' +str(int(len(data['plot' + str(plot_num)]['data']['ys'])))+ ' entries. '

    # answer
    la = []
    for k,v in data['plot'+str(plot_num)]['data']['data params'].items():
        la.append(v)

    a = {big_tag_short + ' ' + adder:la}

    if verbose:
        print('QUESTION:', q)
        print('ANSWER:', a)
    if return_qa: 
        if big_tag_short + ' ' + adder not in qa_pairs['Level 3']['Plot-level questions']:
            qa_pairs['Level 3']['Plot-level questions'][big_tag_short + ' ' + adder] = {'plot'+str(plot_num):{'Q':q, 'A':a, 
                                                                                                              'note':'this currently assumes all elements on a single plot have the same relationship type'}}
        else:
            qa_pairs['Level 3']['Plot-level questions'][big_tag_short + ' ' + adder]['plot'+str(plot_num)] = {'Q':q, 'A':a, 
                                                                                                              'note':'this currently assumes all elements on a single plot have the same relationship type'}
        return qa_pairs

In [65]:
if datas['plot'+str(plot_num)]['type'] == 'line':
    hasLine = False
    if 'data params' not in datas['plot'+str(iplot)]['data']:
        print('Not a gmm or linear relationship!')
    else:
        for k,v in datas['plot'+str(iplot)]['data']['data params'].items():
            if 'line' in k:
                hasLine = True
    if hasLine:
        q_linear_gmm_lines(datas, qa_pairs, plot_num=iplot, return_qa=False, use_words=False, use_nlines = False)
        print('')
        q_linear_gmm_lines(datas, qa_pairs, plot_num=iplot, return_qa=False, use_words=True, use_nlines = False)
        print('')
        print('')
        q_linear_gmm_lines(datas, qa_pairs, plot_num=iplot, return_qa=False, use_words=False, use_nlines = True)
        print('')
        q_linear_gmm_lines(datas, qa_pairs, plot_num=iplot, return_qa=False, use_words=True, use_nlines = True)

Not a gmm or linear relationship!


## PLOT with QA's

In [ ]:
plot_params_here = plot_params_line.copy()

ifigure = 0

datas_all = {} # for multiples
plt.close('all')

verbose_qa = False

#if True:
for ifigure in range(10):
    ######### pick things ########
    # figure
    color_map = 'gray' # hack#np.random.choice(plt.colormaps())
    #plt.set_cmap(color_map)
    
    npanels, panel_style, nrows, ncols = get_nrows_and_ncols(panel_params)
    ####npanels, panel_style,nrows,ncols = 1, 'horizontal', 1,1 # HACK
    plot_style = np.random.choice(plot_styles)
    
    aspect_fig = np.random.uniform(low=aspect_fig_params['min'], 
                                   high=aspect_fig_params['max'])
    # rows and cols 
    aspect_fig *= nrows/ncols
    dpi = int(np.random.uniform(low=dpi_params['min'], high=dpi_params['max']))
    # tight layout or not
    print('panel style:', panel_style, 'nrows,ncols=', (nrows,ncols), 
          'aspect=', aspect_fig, 'dpi=', dpi)
    
    tight_layout = True


    # get all font stuffs
    title_fontsize, xlabel_fontsize, ylabel_fontsize, \
       xlabel_ticks_fontsize, ylabel_ticks_fontsize, \
                           csfont = get_font_info(fontsizes, font_names)
    #print("ALL IS WELL HERE 1")

    # plot types?
    success = False
    start_time = time.time()
    while not success:
        #with plt.style.context(plot_style):
        if True:
            plt.style.use(plot_style)
            plt.set_cmap(color_map) 
            #plt.rcParams['image.cmap'] = 'gray' #color_map
            if tight_layout:
                fig,axes = plt.subplots(nrows,ncols,figsize=(base*nrows, base*aspect_fig*ncols), 
                                        dpi=dpi,layout='tight')
            else:
                fig,axes = plt.subplots(nrows,ncols,figsize=(base*nrows, base*aspect_fig*ncols), 
                                        dpi=dpi)
                
            if npanels == 1:
                axes = [axes]
                plot_inds = [(0,0)] # i,j
            else: # flatten, for now
                # create inds
                if len(axes.shape) > 1: # 2d
                    ashape = np.array(axes.shape).copy()
                else:
                    ashape = [nrows, ncols]
                plot_inds = np.empty(shape=(ashape[0], ashape[1],2), dtype=int)
                for i in range(ashape[0]):
                    for j in range(ashape[1]):
                        plot_inds[i,j][0] = i
                        plot_inds[i,j][1] = j
                plot_inds = plot_inds.reshape((ashape[0]*ashape[1],-1))
                axes = axes.flatten()
            

            #**HERE** have to save index
                    
            ######### Generate plot data #########
            #print('here 1')
            
            choices = []; probs = []
            for k,v in plot_params_here.items():
                choices.append(k)
                probs.append(v['prob'])
        
            data_for_plots = []
            plot_types = []
            data_from_plots = []
            titles = []; xlabels = []; ylabels = []; cbars = []
            end_time = time.time()
            distribution_types = []
            #try:
            if True:
                for iplot, ax in enumerate(axes):
                    plot_params_here_ax = plot_params_here.copy()
                    start_time = time.time()
                    plot_type = np.random.choice(choices, p=probs)
                    print('PLOT TYPE:', plot_type)
                    # get distribution type
                    dist_params = plot_params_here_ax[plot_type]['distribution'] 
                    choices_d = []; probs_d = []
                    for k,v in dist_params.items():
                        choices_d.append(k)
                        probs_d.append(v['prob'])

                    distribution_type = np.random.choice(choices_d, p=probs_d)
                    print('  Distribution Type:', distribution_type)
                    
                    xmin,xmax = log_scale_ax()
                    plot_params_here_ax[plot_type]['xmin']=xmin
                    plot_params_here_ax[plot_type]['xmax']=xmax
                    if plot_type == 'line' or plot_type == 'scatter' or plot_type == 'contour':
                        ymin,ymax = log_scale_ax()
                        plot_params_here_ax[plot_type]['ymin']=ymin
                        plot_params_here_ax[plot_type]['ymax']=ymax
                    if plot_type == 'scatter' or plot_type == 'contour': 
                        cmin,cmax = log_scale_ax()
                        plot_params_here_ax[plot_type]['color']['min']=cmin
                        plot_params_here_ax[plot_type]['color']['max']=cmax
  
                    
                    #print('here 2')
                    data_for_plot = get_data(plot_params_here_ax[plot_type],
                                    plot_type=plot_type,
                                            distribution=distribution_type)
                    end_time = time.time()
                
                    ######### PLOT ############
                    start_time = time.time()
                    #print('here 3')
                    data_from_plot, ax = make_plot(plot_params_here_ax[plot_type], data_for_plot, 
                                         ax, plot_type=plot_type, linestyles=linestyles)#, plot_style=plot_style)
                    #import sys; sys.exit()
                    end_time = time.time()
                    #print('here 4')
    
                    # set ticksizes
                    ax.tick_params(axis='x', which='major', labelsize=xlabel_ticks_fontsize, labelfontfamily=csfont['fontname'])
                    ax.tick_params(axis='y', which='major', labelsize=ylabel_ticks_fontsize, labelfontfamily=csfont['fontname'])
    
                    start_time = time.time()
                    p = np.random.uniform(0,1)
                    if p < title_params['prob']:
                        title_words = get_titles_or_labels(popular_nouns, title_params['capitalize'],
                                                 title_params['equation'], inlines,
                                                 nwords=np.random.randint(low=title_params['n words']['min'],
                                                                          high=title_params['n words']['max']+1))
                        title = ax.set_title(title_words, fontsize = title_fontsize, **csfont)
                    else:
                        title = ''
                        
                    xlabel_words = get_titles_or_labels(popular_nouns, xlabel_params['capitalize'],
                                                 xlabel_params['equation'], inlines,
                                                 nwords=np.random.randint(low=xlabel_params['n words']['min'],
                                                                          high=xlabel_params['n words']['max']+1))
                    xlabel = ax.set_xlabel(xlabel_words, fontsize=xlabel_fontsize, **csfont)
                    ylabel_words = get_titles_or_labels(popular_nouns, ylabel_params['capitalize'],
                                                 ylabel_params['equation'], inlines,
                                                 nwords=np.random.randint(low=ylabel_params['n words']['min'],
                                                                          high=ylabel_params['n words']['max']+1))
                    ylabel = ax.set_ylabel(ylabel_words, fontsize=ylabel_fontsize, **csfont)
                    end_time = time.time()
            
                    # save
                    data_for_plots.append(data_for_plot)
                    plot_types.append(plot_type)
                    data_from_plots.append(data_from_plot)
                    titles.append(title)
                    xlabels.append(xlabel)
                    ylabels.append(ylabel)
                    distribution_types.append(distribution_type)
    
                    if plot_type == 'scatter': # or plot_type == 'contour':
                        if 'color bar' in data_from_plot:
                            side = data_from_plot['color bar params']['side']
                            if side == 'top' or side == 'bottom':
                                orientation = 'horizontal'
                            else:
                                orientation = 'vertical'
    
                            cbar = fig.colorbar(data_from_plot['data'], 
                                         cax=data_from_plot['color bar'], 
                                         orientation=orientation)
                            cbars.append(cbar)

                    if plot_type == 'contour':
                        if 'color bar' in data_from_plot:
                            side = data_from_plot['color bar params']['side']
                            if side == 'top' or side == 'bottom':
                                orientation = 'horizontal'
                            else:
                                orientation = 'vertical'
                    
                            if 'image' in data_from_plot['data']: # select correct colorbar to use
                                datac = data_from_plot['data']['image']
                            else:
                                datac = data_from_plot['data']['contour']
                                            
                            cbar = fig.colorbar(datac, 
                                         cax=data_from_plot['color bar'], 
                                         orientation=orientation)
                            cbars.append(cbar)

    
                #if not save_smalls:
                plt.set_cmap(color_map) # do again
                fig.savefig(fake_figs_dir + 'Picture' + str(ifigure+1) + '.png', dpi=dpi, bbox_inches='tight')
                print('saved:', fake_figs_dir + 'Picture' + str(ifigure+1) + '.png')
                # else:
                #     fig.savefig(save_small_dir + 'Picture' + str(ifigure+1) + '.png', dpi=dpi)
                #     print('saved:', save_small_dir + 'Picture' + str(ifigure+1) + '.png')
                    
                success = True
            #except Exception as e:
            else:
                print('[ERROR]:')
                plt.close(fig)
                print('issue with plotting, trying again')
                print(e)
                if 'unknown color specifier' in str(e):
                    print(e)
                    import sys; sys.exit()
                if 'At least one value in the dash list must be positive' in str(e):
                    print(e)
                    print(data_from_plots)
    
    
    ####### end of plotting ############
    # try the whole thing again
    width, height = fig.canvas.get_width_height()
    # save data
    datas = {}
    # figure datas
    datas['figure'] = {'dpi':dpi, 'base':base, 'aspect ratio': aspect_fig, 
                          'nrows':nrows, 'ncols':ncols, 
                         'plot style':plot_style, 
                         'color map':color_map,
                         'title fontsize':title_fontsize, 
                         'xlabel fontsize':xlabel_fontsize,
                         'ylabel fontsize':ylabel_fontsize, 
                      'plot indexes':plot_inds}
    
    # now, get data things
    for iplot, ax in enumerate(axes):
        ###### get data from plot ######
        data_from_plot = data_from_plots[iplot]
        data_for_plot = data_for_plots[iplot]
        plot_type = plot_types[iplot]
        title = titles[iplot]
        xlabel = xlabels[iplot]
        ylabel = ylabels[iplot]
    
        data_pixels = get_data_pixel_locations(data_from_plot, plot_type, ax, width, height)
    
        # bounding box of square
        bbox = ax.get_position() # Bbox(x0, y0, x1, y1)
        xpix1 = np.array([bbox.x0,bbox.x1])
        ypix1 = np.array([bbox.y0,bbox.y1])
        xpix1 *= width
        ypix1 *= height
        
        # x-tick locations
        xticks = get_ticks(ax.get_xticklabels(), ax.get_xticklines())
    
        # y-tick locations
        yticks = get_ticks(ax.get_yticklabels(), ax.get_yticklines())
        
        # for colorbars
        colorbar_ticks = []
        if 'color bar' in data_from_plots[iplot]:
            colorbar = data_from_plots[iplot]['color bar']
            if data_from_plots[iplot]['color bar params']['side'] == 'left' \
               or data_from_plots[iplot]['color bar params']['side'] == 'right':
                ticks = colorbar.get_yticklabels()
                tick_locs = colorbar.get_yticklines(minor=False)
            else:
                ticks = colorbar.get_xticklabels()
                tick_locs = colorbar.get_xticklines(minor=False)
            colorbar_ticks = get_ticks(ticks, tick_locs)

        # title
        # Get the bounding box of the title in display space
        if title != '':
            title_bbox = title.get_window_extent()
            title_words = title.get_text()
        else:
            title_bbox = -1
            title_words = ''
    
        # xlabel
        xlabel_bbox = xlabel.get_window_extent()
        xlabel_words = xlabel.get_text()
        # ylabel
        ylabel_bbox = ylabel.get_window_extent()
        ylabel_words = ylabel.get_text()
    
    
        ########## save data ##############

        # line plot 
        plot_name = 'plot' + str(iplot) 
        datas[plot_name] = {}
        # line plot type
        datas[plot_name]['type'] = plot_type # tag for kind of plot
        datas[plot_name]['distribution'] = distribution_types[iplot]
        datas[plot_name]['data'] = data_for_plot
        datas[plot_name]['data pixels'] = data_pixels
        datas[plot_name]['data from plot'] = json.loads(json.dumps(data_from_plot, cls=NumpyEncoder))
        if (plot_type == 'scatter' or plot_type == 'contour') and 'color bar' in data_from_plots[iplot]:
            #print('yes indeed')
            w = data_from_plots[iplot]['color bar'].get_window_extent()
            datas[plot_name]['color bar'] = {'xmin':w.x0,'ymin':w.y0,
                                             'xmax':w.x1,'ymax':w.y1, 
                                             'params':data_from_plot['color bar params']}
        xtmp = []
        for xt in xticks:
            l = {'data':xt[0], 'xmin': xt[1], 
                 'ymin': xt[2], 
                 'xmax':xt[3], 'ymax':xt[4],
                 'tx':xt[5], 'ty':xt[6]}
            xtmp.append(l)
        datas[plot_name]['xticks'] = xtmp.copy()
        # 
        xtmp = []
        for xt in yticks:
            l = {'data':xt[0], 'xmin': xt[1], 
                 'ymin': xt[2], 
                 'xmax':xt[3], 'ymax':xt[4], 
                'tx':xt[5], 'ty':xt[6]}
            xtmp.append(l)
        datas[plot_name]['yticks'] = xtmp.copy()
        if len(colorbar_ticks) > 0:
            xtmp = []
            for xt in colorbar_ticks:
                l = {'data':xt[0], 'xmin': xt[1], 
                     'ymin': xt[2], 
                     'xmax':xt[3], 'ymax':xt[4], 
                    'tx':xt[5], 'ty':xt[6]}
                xtmp.append(l)
            datas[plot_name]['color bar ticks'] = xtmp.copy()
            
        # axis box
        datas[plot_name]['square'] = {'xmin':xpix1[0], 'ymin':ypix1[0], 
                                         'xmax':xpix1[1], 'ymax':ypix1[1]}
        # title
        if title_bbox != -1:
            datas[plot_name]['title'] = {'xmin':title_bbox.x0, 'ymin':title_bbox.y0, 
                                            'xmax':title_bbox.x1, 'ymax':title_bbox.y1,
                                           'words':title_words}
        else:
            pass
        datas[plot_name]['xlabel'] = {'xmin':xlabel_bbox.x0, 'ymin':xlabel_bbox.y0, 
                                        'xmax':xlabel_bbox.x1, 'ymax':xlabel_bbox.y1,
                                       'words':xlabel_words}
        datas[plot_name]['ylabel'] = {'xmin':ylabel_bbox.x0, 'ymin':ylabel_bbox.y0, 
                                        'xmax':ylabel_bbox.x1, 'ymax':ylabel_bbox.y1,
                                       'words':ylabel_words}

    # dump full data
    dumped = json.dumps(datas, cls=NumpyEncoder)
    with open(fake_figs_dir + 'Picture' + str(ifigure+1) + '_fullData.json', 'w') as f:
        json.dump(dumped, f)

    # create qa pairs
    qa_pairs = {}
    # question levels    
    qa_pairs['Level 1'] = {}
    qa_pairs['Level 2'] = {}
    qa_pairs['Level 3'] = {}
    qa_pairs['Level 1']['Figure-level questions'] = {} # Figure-level questions
    qa_pairs['Level 1']['Plot-level questions'] = {}
    qa_pairs['Level 2']['Plot-level questions'] = {}
    qa_pairs['Level 3']['Plot-level questions'] = {}

    #print(qa_pairs)

    ###### L1 ######
    # basic figure level
    qa_pairs = q1(datas, qa_pairs, verbose=verbose_qa)
    qa_pairs = q2(datas, qa_pairs, verbose=verbose_qa)
    qa_pairs = q3(datas, qa_pairs, verbose=verbose_qa)
    qa_pairs = q4(datas, qa_pairs, verbose=verbose_qa)

    # basic, plot-level
    # titles/axis labels
    qa_pairs = q_plot_titles(datas, qa_pairs, verbose=verbose_qa)
    qa_pairs = q_plot_xlabels(datas, qa_pairs, verbose=verbose_qa)
    qa_pairs = q_plot_ylabels(datas, qa_pairs, verbose=verbose_qa)
    # x/y axis tick labels
    qa_pairs = ticklabels(datas, qa_pairs, tick_type='x', verbose=verbose_qa)
    qa_pairs = ticklabels(datas, qa_pairs, tick_type='y', verbose=verbose_qa)
    # plot types
    qa_pairs = q_plot_types(datas, qa_pairs, plot_types_qa, use_list=True, verbose=verbose_qa)

    # line-plot specifics
    for iplot in range(len((axes))):
        if datas['plot'+str(iplot)]['type'] == 'line': # line plots
            ####### L1 ########
            # line plots in numbers/words
            qa_pairs = q_nlines_plot_plotnums(datas, qa_pairs, plot_num = iplot, verbose=verbose_qa)
            qa_pairs = q_nlines_plot(datas, qa_pairs, plot_num = iplot, verbose=verbose_qa)
            # colors of lines
            qa_pairs = q_colors_lines(datas, qa_pairs, plot_num=iplot, use_words=False, verbose=verbose_qa)
            qa_pairs = q_colors_lines(datas, qa_pairs, plot_num=iplot, use_words=True, verbose=verbose_qa)
            # linestyles
            qa_pairs = q_linestyles_lines(datas, qa_pairs, plot_num=iplot, 
                                          use_words=False, use_list=False, verbose=verbose_qa)
            qa_pairs = q_linestyles_lines(datas, qa_pairs, plot_num=iplot, 
                                          use_words=True, use_list = False, verbose=verbose_qa)
            qa_pairs = q_linestyles_lines(datas, qa_pairs, plot_num=iplot, 
                                          use_words=False, use_list=True,
                              linestyle_list=linestyles, verbose=verbose_qa)
            qa_pairs = q_linestyles_lines(datas, qa_pairs, plot_num=iplot, 
                                          use_words=True, use_list = True,
                              linestyle_list=linestyles, verbose=verbose_qa)
            # line thicknesses
            qa_pairs = q_linethickness_lines(datas, qa_pairs, 
                                             plot_num=iplot, use_words=False, verbose=verbose_qa)
            qa_pairs = q_linethickness_lines(datas, qa_pairs, plot_num=iplot,  
                                             use_words=True, verbose=verbose_qa)
            # line markers
            qa_pairs = q_linemarkers_lines(datas, qa_pairs, plot_num=iplot, use_words=False, verbose=verbose_qa)
            qa_pairs = q_linemarkers_lines(datas, qa_pairs, plot_num=iplot, use_words=True, verbose=verbose_qa)
            # line marker sizes
            qa_pairs = q_linemarkersize_lines(datas, qa_pairs, plot_num=iplot, use_words=False, verbose=verbose_qa)
            qa_pairs = q_linemarkersize_lines(datas, qa_pairs, plot_num=iplot, use_words=True, verbose=verbose_qa)

            ###### L2 #######
            # stats items
            for k,v in stats.items(): # for all stats
                qa_pairs = q_stats_lines(datas, qa_pairs, stat={k:v}, 
                                         plot_num=iplot, use_words=False, verbose=verbose_qa)
                qa_pairs = q_stats_lines(datas, qa_pairs, stat={k:v}, 
                                         plot_num=iplot, use_words=True, verbose=verbose_qa)

            # existance of error bars & values
            for axisv in ['x','y']:
                qa_pairs = q_errorbars_existance_lines(datas, qa_pairs, 
                                                       axis=axisv, plot_num=iplot, use_words=False, verbose=verbose_qa)
                qa_pairs = q_errorbars_existance_lines(datas, qa_pairs, 
                                                       axis=axisv, plot_num=iplot, use_words=True, verbose=verbose_qa)  

                if axisv + 'errs' in datas['plot'+str(iplot)]['data']:
                    qa_pairs = q_errorbars_size_lines(datas, qa_pairs, axis=axisv, 
                                                      plot_num=iplot, use_words=False, verbose=verbose_qa)
                    qa_pairs = q_errorbars_size_lines(datas, qa_pairs, axis=axisv, 
                                                      plot_num=iplot, use_words=True, verbose=verbose_qa) 

            ###### L3 ######
            # type of functional relationship
            qa_pairs = q_relationship_lines(datas, qa_pairs, plot_num=iplot, use_words=False, use_nlines = False, verbose=verbose_qa)
            qa_pairs = q_relationship_lines(datas, qa_pairs, plot_num=iplot, use_words=True, use_nlines = False, verbose=verbose_qa)
            qa_pairs = q_relationship_lines(datas, qa_pairs, plot_num=iplot, use_words=False, use_nlines = True, verbose=verbose_qa)
            qa_pairs = q_relationship_lines(datas, qa_pairs, plot_num=iplot, use_words=True, use_nlines = True, verbose=verbose_qa)
            # linear parameters
            hasLine = False
            hasGMM = False
            if 'data params' not in datas['plot'+str(iplot)]['data']:
                if verbose_qa: print('Not a linear relationship!')
            else:
                for k,v in datas['plot'+str(iplot)]['data']['data params'].items():
                    if 'line' in k and distribution_types[iplot]=='linear':
                        hasLine = True
                    elif 'line' in k and distribution_types[iplot]=='gmm':
                        hasGMM = True
            if hasLine:
                qa_pairs = q_linear_equation_lines(datas, qa_pairs, plot_num=iplot, 
                                                   use_words=False, use_nlines = False, verbose=verbose_qa)
                qa_pairs = q_linear_equation_lines(datas, qa_pairs, plot_num=iplot, 
                                                   use_words=True, use_nlines = False, verbose=verbose_qa)
                qa_pairs = q_linear_equation_lines(datas, qa_pairs, plot_num=iplot, 
                                                   use_words=False, use_nlines = True, verbose=verbose_qa)
                qa_pairs = q_linear_equation_lines(datas, qa_pairs, plot_num=iplot, 
                                                   use_words=True, use_nlines = True, verbose=verbose_qa)

            if hasGMM:
                qa_pairs = q_linear_gmm_lines(datas, qa_pairs, plot_num=iplot, 
                                              use_words=False, use_nlines = False, verbose=verbose_qa)
                qa_pairs = q_linear_gmm_lines(datas, qa_pairs, plot_num=iplot, 
                                              use_words=True, use_nlines = False, verbose=verbose_qa)
                qa_pairs = q_linear_gmm_lines(datas, qa_pairs, plot_num=iplot, 
                                              use_words=False, use_nlines = True, verbose=verbose_qa)
                qa_pairs = q_linear_gmm_lines(datas, qa_pairs, plot_num=iplot, 
                                              use_words=True, use_nlines = True, verbose=verbose_qa)  
                
    # also dump qa
    dumped = json.dumps(qa_pairs, cls=NumpyEncoder)
    with open(fake_figs_dir + 'Picture' + str(ifigure+1) + '_qa.json', 'w') as f:
        json.dump(dumped, f)
            
    # for full thing
    #datas_all[ifigure] = datas.copy()
    plt.close(fig)
print('DONE!')
# back to basics
plt.style.use('default')

In [ ]:
fig

In [142]:
# read in one
ifigure = 2
with open(fake_figs_dir + 'Picture' + str(ifigure+1) + '_qa.json', 'r') as f:
    qa_pairs_in = json.loads(f.read())
    qa_pairs_in = json.loads(qa_pairs_in)

In [143]:
qa_pairs_in['Level 3']['Plot-level questions']

{'relationship (plot numbers + list)': {'plot0': {'Q': 'The following question refers to the figure panel on row number 0 and column number 0. If there are multiple plots the panels will be in row-major (C-style) order, with the numbering starting at (0,0) in the upper left panel. If there is one plot, then this row and column refers to the single plot. What is the functional relationship between the x and y values for the figure panel on row number 0 and column number 0? You are a helpful assistant, please format the output as a json as {"relationship":[]} where each element of the list corresponds to a single line in the figure. Please choose each relationship for each line from the following list for each line: [random, linear, gaussian mixture model].',
   'A': {'relationship (plot numbers + list)': ['gaussian mixture model',
     'gaussian mixture model',
     'gaussian mixture model',
     'gaussian mixture model',
     'gaussian mixture model']},
   'note': 'this currently assum

In [144]:
qa_pairs_in

{'Level 1': {'Figure-level questions': {'rows/columns': {'Q': 'How many panels are in this figure? You are a helpful assistant, please format the output as a json as {"nrows":"", "ncols":""} to store the number of rows and columns.',
    'A': {'nrows': 1, 'ncols': 3}},
   'plot style': {'Q': 'Assuming this is a figure made with matplotlib in Python, what is the plot style used?  Examples of plotting styles are "classic" or "ggplot". You are a helpful assistant, please format the output as a json as {"plot style":""} to store the matplotlib plotting style used in the figure.',
    'A': {'plot style': 'seaborn-v0_8-muted'}},
   'colormap': {'Q': 'Assuming this is a figure made with matplotlib in Python, what is the colormap that was used?  Examples of matplotlib colormaps are "rainbow" or "Reds". You are a helpful assistant, please format the output as a json as {"colormap":""} to store the matplotlib colormap used in the figure.',
    'A': {'colormap': 'gray'},
    'notes': "Some of the